In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install -U bertopic[visualization]

In [ ]:
!pip install bertopic umap-learn hdbscan sentence-transformers

In [ ]:
!pip install -U bertopic[visualization] umap-learn hdbscan sentence-transformers gensim nltk textblob networkx matplotlib scikit-learn

In [ ]:
#spacy

!pip -q install spacy
!python -m spacy download en_core_web_trf

import spacy
from spacy.pipeline import EntityRuler


In [ ]:
# Calculate token statistics

from transformers import AutoTokenizer
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-base-en-v1.5")

# Read data
df = pd.read_csv("/content/drive/MyDrive/wired_project/wired_2014_2024_cleaned.csv")
texts = df["text"].dropna().tolist()

# Compute token length and truncation ratio
max_len = 512
token_lens = []
percent_losses = []

for text in texts:
    tokens = tokenizer.encode(text, truncation=False)
    total_len = len(tokens)
    truncated_len = min(total_len, max_len)
    loss = max(total_len - max_len, 0)
    percent_loss = (loss / total_len) * 100 if total_len > 0 else 0

    token_lens.append(total_len)
    percent_losses.append(percent_loss)

# Visualize distribution of truncation percentage
plt.figure(figsize=(10, 5))
plt.hist(percent_losses, bins=40, color="skyblue", edgecolor="black")
plt.xlabel("Percentage Loss")
plt.ylabel("Number of Documents")
plt.title("Distribution of Percentage Loss by BGE Tokenizer (max 512 tokens)")
plt.grid(True)
plt.show()

# Output basic statistics
percent_losses_np = np.array(percent_losses)
print(f"Average truncation ratio: {percent_losses_np.mean():.2f}%")
print(f"Median truncation ratio: {np.median(percent_losses_np):.2f}%")
print(f"Number of documents with 0% truncation: {(percent_losses_np == 0).sum()} / {len(percent_losses_np)}")
print(f"Number of documents with truncation ratio > 60%: {(percent_losses_np > 60).sum()} / {len(percent_losses_np)}")


In [ ]:
import os
import time
import numpy as np
import pandas as pd
from tqdm import tqdm
from sentence_transformers import SentenceTransformer

# Config
CSV_IN       = "wired_2014_2024_cleaned.csv"
TEXT_COL     = "text"
MODEL_NAME   = "BAAI/bge-base-en-v1.5"
MAX_LEN      = 512
BATCH_SIZE   = 64
OUT_MEMMAP   = "wired_bge-base-v1.5_len512.memmap"
NORMALIZE    = True

print(f"Loading data: {CSV_IN}")
df = pd.read_csv(CSV_IN)
texts = df[TEXT_COL].astype(str).tolist()
N = len(texts)
print(f"Articles: {N}")

print(f"Loading model: {MODEL_NAME}")
model = SentenceTransformer(MODEL_NAME)

dummy = model.encode(["test"], normalize_embeddings=NORMALIZE, truncation=True, max_length=32)
DIM = dummy.shape[1]
print(f"Embedding dim: {DIM}")

if os.path.exists(OUT_MEMMAP):
    emb = np.memmap(OUT_MEMMAP, dtype="float16", mode="r+", shape=(N, DIM))
    zero_rows = np.where(np.all(emb == 0, axis=1))[0]
    pos = int(zero_rows[0]) if len(zero_rows) > 0 else N
    print(f"Resuming from position {pos}/{N}")
else:
    emb = np.memmap(OUT_MEMMAP, dtype="float16", mode="w+", shape=(N, DIM))
    pos = 0
    print("Starting fresh run...")

# Main loop
start_time = time.time()
for i in tqdm(range(pos, N, BATCH_SIZE), desc="Generating embeddings", unit="batch"):
    batch = texts[i:i+BATCH_SIZE]
    vec = model.encode(
        batch,
        batch_size=BATCH_SIZE,
        normalize_embeddings=NORMALIZE,
        truncation=True,
        max_length=MAX_LEN,
        show_progress_bar=False
    ).astype(np.float16)
    emb[i:i+len(batch)] = vec
    emb.flush()

    elapsed = time.time() - start_time
    done = i + len(batch) - pos
    total = N - pos
    speed = done / elapsed
    eta = (total - done) / speed if speed > 0 else 0
    tqdm.write(f"Batch {i//BATCH_SIZE+1}, speed={speed:.2f} docs/s, ETA={eta/60:.1f} min")

emb.flush()
print(f"Saved memmap: {OUT_MEMMAP}  shape=({N}, {DIM})  dtype=float16")

# Save metadata
meta = {
    "model": MODEL_NAME,
    "max_length": MAX_LEN,
    "batch_size": BATCH_SIZE,
    "normalize": NORMALIZE,
    "n_articles": N,
    "dim": int(DIM)
}
pd.Series(meta).to_json("embedding_meta.json", indent=2)
print("Wrote embedding_meta.json")


In [ ]:
# Load article texts
df = pd.read_csv(CSV_PATH)
docs_all = df["text"].dropna().astype(str).tolist()
N_TOTAL = len(docs_all)
EMB_DIM = 768

print(f"Total documents: {N_TOTAL:,}")

#Delete old file if exists
if os.path.exists(EMBED_PATH):
    os.remove(EMBED_PATH)
    print("Old memmap file deleted")

#Create memmap file and write embeddings
print("Loading BGE model...")
model = SentenceTransformer("BAAI/bge-base-en-v1.5")

print("Generating embeddings and saving as memmap...")
batch_size = 32
embeddings = np.memmap(EMBED_PATH, dtype=np.float32, mode='w+', shape=(N_TOTAL, EMB_DIM))

for i in tqdm(range(0, N_TOTAL, batch_size)):
    batch = docs_all[i:i+batch_size]
    batch_emb = model.encode(batch, show_progress_bar=False)
    embeddings[i:i+len(batch_emb)] = batch_emb

embeddings.flush()
print(f"Memmap file saved as: {EMBED_PATH}")
print(f"File shape: {embeddings.shape}")

In [ ]:
CSV_PATH = "/content/drive/MyDrive/wired_project/wired_2014_2024_cleaned.csv"
EMBED_PATH = "/content/drive/MyDrive/wired_project/wired_bge-base-v1.5_len512.memmap"
OUT_DIR = "/content/drive/MyDrive/wired_project"
DIM = 768

final_params = {
    "min_cluster_size": 100,
    "min_samples": 15,
    "n_neighbors": 12,
    "min_dist": 0.02,
    "n_components": 5,
    "nr_topics": "auto",
    "top_n_words": 10,
    "compute_npmi": True,
    "npmi_docs": 2000
}

# Load data and embeddings
print("Loading texts...")
df = pd.read_csv(CSV_PATH)
docs_all = df["text"].dropna().astype(str).tolist()
print(f"Number of documents: {len(docs_all):,}")

print("Loading embeddings...")
emb_all = np.memmap(EMBED_PATH, dtype=np.float32, mode="r", shape=(len(docs_all), DIM))
print(f"Embedding matrix shape: {emb_all.shape}")

print("\nTraining BERTopic model...")

umap_model = UMAP(
    n_neighbors=final_params["n_neighbors"],
    n_components=final_params["n_components"],
    min_dist=final_params["min_dist"],
    metric="cosine",
    random_state=42
)

hdbscan_model = HDBSCAN(
    min_cluster_size=final_params["min_cluster_size"],
    min_samples=final_params["min_samples"],
    metric="euclidean",
    prediction_data=True
)

topic_model = BERTopic(
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    top_n_words=final_params["top_n_words"],
    nr_topics=final_params["nr_topics"],
    calculate_probabilities=False,
    verbose=True
)

# Train model
topics, probs = topic_model.fit_transform(docs_all, embeddings=emb_all)

# Calculate NPMI
print("Calculating NPMI coherence...")

try:
    npmi_docs = docs_all[:final_params["npmi_docs"]]
    vectorizer = topic_model.vectorizer_model
    analyzer = vectorizer.build_analyzer()
    tokenized_docs = [analyzer(doc) for doc in npmi_docs]
    dictionary = corpora.Dictionary(tokenized_docs)
    corpus = [dictionary.doc2bow(tokens) for tokens in tokenized_docs]

    topics_words = []
    for topic_id in range(len(topic_model.get_topics())):
        topic_words = topic_model.get_topic(topic_id)
        if topic_words:
            words = [w for w, _ in topic_words[:10]]
            topics_words.append(words)

    cm = CoherenceModel(
        topics=topics_words,
        texts=tokenized_docs,
        dictionary=dictionary,
        corpus=corpus,
        coherence='c_npmi'
    )
    c_npmi_score = cm.get_coherence()
    print(f"NPMI coherence: {c_npmi_score:.4f}")
except Exception as e:
    print(f"NPMI calculation failed: {e}")
    c_npmi_score = float("nan")

# Calculate topic diversity
def calculate_topic_diversity(topic_model, topk=10):
    topics = topic_model.get_topics()
    unique_words = set()
    total_words = 0
    for topic_id in topics:
        if topic_id == -1:
            continue
        words = [word for word, _ in topics[topic_id][:topk]]
        unique_words.update(words)
        total_words += len(words)
    return len(unique_words) / total_words if total_words > 0 else 0

# Save model and metrics
print("Saving model and results...")

topic_model.save(f"{OUT_DIR}/bertopic_final_model_v3")
topic_model.get_topic_info().to_csv(f"{OUT_DIR}/bertopic_final_topics_v3.csv", index=False)
topic_model.get_document_info(docs_all).to_csv(f"{OUT_DIR}/bertopic_final_docinfo_v3.csv", index=False)

final_metrics = {
    "n_topics": len(topic_model.get_topics()) - 1,
    "outlier_rate": float(np.array(topics == -1).mean()),
    "topic_diversity": calculate_topic_diversity(topic_model),
    "c_npmi": c_npmi_score,
    **final_params
}

with open(f"{OUT_DIR}/bertopic_final_metrics_v3.json", "w") as f:
    json.dump(final_metrics, f, indent=2)

# Show results
print("\nFull model training completed. Results saved in:", OUT_DIR)
print("Number of topics:", final_metrics["n_topics"])
print("NPMI:", final_metrics["c_npmi"])
print("Outlier rate:", final_metrics["outlier_rate"])
print("Diversity:", final_metrics["topic_diversity"])

In [ ]:
# Load data
import pandas as pd

CSV_PATH = "/content/drive/MyDrive/wired_project/wired_2014_2024_cleaned.csv"
df = pd.read_csv(CSV_PATH)
docs = df["text"].dropna().astype(str).tolist()

# Download punkt
import nltk
nltk.download('punkt')

#Word frequency statistics
from nltk.tokenize import word_tokenize
from collections import Counter
import matplotlib.pyplot as plt

flat_tokens = []
for doc in docs[:30000]:
    try:
        words = word_tokenize(doc.lower())
        words = [w for w in words if w.isalpha()]
        flat_tokens.extend(words)
    except Exception as e:
        print("Text error, skipped:", e)
        continue

counter = Counter(flat_tokens)
common_words = counter.most_common(100)

# Plot bar chart
df_common = pd.DataFrame(common_words, columns=["word", "freq"])
plt.figure(figsize=(12, 6))
plt.bar(df_common["word"], df_common["freq"])
plt.xticks(rotation=90)
plt.title("Top 100 Frequent Words in Wired Corpus")
plt.tight_layout()
plt.show()


In [ ]:
# Word frequency statistics
# Get Top 100 frequent words
import pandas as pd
from collections import Counter
import re

CSV_PATH = "/content/drive/MyDrive/wired_project/wired_2014_2024_cleaned.csv"
df = pd.read_csv(CSV_PATH)
docs = df["text"].dropna().astype(str).tolist()

print(f"Loaded {len(docs):,} documents")

def get_top_words(docs, top_n=100, sample_size=10000):
    """Get the most frequent words"""
    flat_tokens = []
    sample_docs = docs[:sample_size] if len(docs) > sample_size else docs
    print(f"Analyzing first {len(sample_docs):,} documents...")

    for i, doc in enumerate(sample_docs):
        if i % 2000 == 0:
            print(f"Progress: {i:,}/{len(sample_docs):,}")
        try:
            words = re.findall(r'\b[a-zA-Z]{2,}\b', doc.lower())
            flat_tokens.extend(words)
        except:
            continue

    counter = Counter(flat_tokens)
    top_words = counter.most_common(top_n)

    print(f"\nTop {top_n} frequent words:")
    print("-" * 30)
    for i, (word, freq) in enumerate(top_words, 1):
        print(f"{i:2d}. {word:<15} {freq:,}")

    return top_words

top_100_words = get_top_words(docs, top_n=100)

print("\nSuggested stopwords list (Python set format):")
print("-" * 50)
words_only = [word for word, freq in top_100_words]
print("custom_stopwords = {")
for i in range(0, len(words_only), 10):
    chunk = words_only[i:i+10]
    formatted_chunk = ", ".join([f"'{word}'" for word in chunk])
    print(f"    {formatted_chunk},")
print("}")


In [ ]:
# Import dependencies
import os, json
import pandas as pd
import numpy as np
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from umap import UMAP
from hdbscan import HDBSCAN
from gensim.models import CoherenceModel
from gensim import corpora
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords', quiet=True)

CSV_PATH = "/content/drive/MyDrive/wired_project/wired_2014_2024_cleaned.csv"
EMBED_PATH = "/content/drive/MyDrive/wired_project/wired_bge-base-v1.5_len512.memmap"
OUT_DIR = "/content/drive/MyDrive/wired_project"
DIM = 768  # embedding dimension

# Custom stopwords
custom_stopwords = {
    'the', 'to', 'of', 'and', 'in', 'that', 'it', 'is', 'for', 'on', 'you',
    'with', 'as', 'this', 'are', 'but', 'be', 'from', 'have', 'they', 'we',
    'at', 'or', 'was', 'an', 'can', 'by', 'has', 'more', 'not', 'like', 'says',
    'he', 'their', 'about', 'if', 'one', 'there', 'so', 'all', 'your', 'its',
    'which', 'people', 'what', 'will', 'up', 'also', 'when', 'out', 'who',
    'just', 're', 'these', 'new', 'some', 'been', 'how', 'our', 'into', 'had',
    'than', 'would', 'his', 'other', 'them', 'time', 'us', 'were', 'could',
    'do', 'now', 'get', 'even', 'she', 'my', 'say', 'says', 'because', 'no',
    'most', 'over', 'wired', 'use', 'make', 'where', 'then', 'only', 'don',
    'those', 'said', 'way', 'may', 'first', 'year', 'think', 'through', 'said',
    'years', 'many'
}

# Merge NLTK stopwords and custom stopwords
nltk_stopwords = set(stopwords.words('english'))
all_stopwords = list(nltk_stopwords.union(custom_stopwords))

print(f"Total stopwords: {len(all_stopwords)}")

final_params = {
    "min_cluster_size": 100,
    "min_samples": 15,
    "n_neighbors": 12,
    "min_dist": 0.02,
    "n_components": 5,
    "nr_topics": "auto",
    "top_n_words": 10,
    "compute_npmi": True,
    "npmi_docs": 10000
}

print("Loading texts...")
df = pd.read_csv(CSV_PATH)
docs_all = df["text"].dropna().astype(str).tolist()
print(f"Number of documents: {len(docs_all):,}")

print("Loading embeddings...")
emb_all = np.memmap(EMBED_PATH, dtype=np.float32, mode="r", shape=(len(docs_all), DIM))
print(f"Embedding matrix shape: {emb_all.shape}")

print("\nTraining BERTopic model...")

umap_model = UMAP(
    n_neighbors=final_params["n_neighbors"],
    n_components=final_params["n_components"],
    min_dist=final_params["min_dist"],
    metric="cosine",
    random_state=42
)

hdbscan_model = HDBSCAN(
    min_cluster_size=final_params["min_cluster_size"],
    min_samples=final_params["min_samples"],
    metric="euclidean",
    prediction_data=True
)

vectorizer_model = CountVectorizer(
    stop_words=all_stopwords,
    ngram_range=(1, 2),
    min_df=10,
    max_df=0.9
)

topic_model = BERTopic(
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    vectorizer_model=vectorizer_model,
    top_n_words=final_params["top_n_words"],
    nr_topics=final_params["nr_topics"],
    calculate_probabilities=False,
    verbose=True
)

topics, probs = topic_model.fit_transform(docs_all, embeddings=emb_all)

print("Calculating NPMI coherence...")

try:
    npmi_docs = docs_all[:final_params["npmi_docs"]]
    vectorizer = topic_model.vectorizer_model
    analyzer = vectorizer.build_analyzer()
    tokenized_docs = [analyzer(doc) for doc in npmi_docs]
    dictionary = corpora.Dictionary(tokenized_docs)
    corpus = [dictionary.doc2bow(tokens) for tokens in tokenized_docs]

    topics_words = []
    for topic_id in range(len(topic_model.get_topics())):
        topic_words = topic_model.get_topic(topic_id)
        if topic_words:
            words = [w for w, _ in topic_words[:10]]
            topics_words.append(words)

    cm = CoherenceModel(
        topics=topics_words,
        texts=tokenized_docs,
        dictionary=dictionary,
        corpus=corpus,
        coherence='c_npmi'
    )
    c_npmi_score = cm.get_coherence()
    print(f"NPMI coherence: {c_npmi_score:.4f}")
except Exception as e:
    print(f"NPMI calculation failed: {e}")
    c_npmi_score = float("nan")

def calculate_topic_diversity(topic_model, topk=10):
    topics = topic_model.get_topics()
    unique_words = set()
    total_words = 0
    for topic_id in topics:
        if topic_id == -1:
            continue
        words = [word for word, _ in topics[topic_id][:topk]]
        unique_words.update(words)
        total_words += len(words)
    return len(unique_words) / total_words if total_words > 0 else 0

print("Saving model and results...")

topic_model.save(f"{OUT_DIR}/bertopic_final_model_v4_stopwords")
topic_model.get_topic_info().to_csv(f"{OUT_DIR}/bertopic_final_topics_v4_stopwords.csv", index=False)
topic_model.get_document_info(docs_all).to_csv(f"{OUT_DIR}/bertopic_final_docinfo_v4_stopwords.csv", index=False)

final_metrics = {
    "n_topics": len(topic_model.get_topics()) - 1,
    "outlier_rate": float(np.array(topics == -1).mean()),
    "topic_diversity": calculate_topic_diversity(topic_model),
    "c_npmi": c_npmi_score,
    **final_params,
    "n_stopwords": len(all_stopwords),
    "n_custom_stopwords": len(custom_stopwords),
    "n_nltk_stopwords": len(nltk_stopwords)
}

with open(f"{OUT_DIR}/bertopic_final_metrics_v4_stopwords.json", "w") as f:
    json.dump(final_metrics, f, indent=2)

print("\nModel with stopwords completed. Results saved in:", OUT_DIR)
print("Number of topics:", final_metrics["n_topics"])
print("NPMI:", final_metrics["c_npmi"])
print("Outlier rate:", final_metrics["outlier_rate"])
print("Diversity:", final_metrics["topic_diversity"])
print("Total stopwords:", final_metrics["n_stopwords"])
print("Custom stopwords:", final_metrics["n_custom_stopwords"])
print("NLTK stopwords:", final_metrics["n_nltk_stopwords"])


In [ ]:
# Calculate Cv coherence
from gensim.models import CoherenceModel
from gensim import corpora

print("Calculating Cv coherence...")

try:
    cv_docs = docs_all[:30000]
    vectorizer = topic_model.vectorizer_model
    analyzer = vectorizer.build_analyzer()
    tokenized_docs = [analyzer(doc) for doc in cv_docs]
    dictionary = corpora.Dictionary(tokenized_docs)

    topics_words = []
    for topic_id in sorted(topic_model.get_topics().keys()):
        if topic_id != -1:
            topic_words = topic_model.get_topic(topic_id)
            words = [w for w, _ in topic_words[:10]]
            topics_words.append(words)

    cm_cv = CoherenceModel(
        topics=topics_words,
        texts=tokenized_docs,
        dictionary=dictionary,
        coherence='c_v'
    )
    cv_score = cm_cv.get_coherence()
    print(f"Cv coherence: {cv_score:.4f}")

except Exception as e:
    print(f"Cv coherence calculation failed: {e}")
    cv_score = float("nan")

In [ ]:
# View top 10 topics

from bertopic import BERTopic
import pandas as pd

model_path = "/content/drive/MyDrive/wired_project/bertopic_final_model_v4_stopwords"

try:
    topic_model = BERTopic.load(model_path)
    print("Model loaded successfully!")

    topic_info = topic_model.get_topic_info()
    print(f"Total number of topics: {len(topic_info)}")
    print("\nTopic info table:")
    print(topic_info)

    topic_info.to_csv('/content/drive/MyDrive/wired_project/topic_info_table.csv', index=False)
    print("\nTopic info table saved to: topic_info_table.csv")

    print("\nDetails of the first 10 topics:")
    for i in range(min(10, len(topic_info))):
        topic_id = topic_info.iloc[i]['Topic']
        if topic_id != -1:
            topic_words = topic_model.get_topic(topic_id)
            count = topic_info.iloc[i]['Count']
            print(f"Topic {topic_id} (Count: {count}):")
            print([word for word, prob in topic_words[:10]])
            print()

except Exception as e:
    print(f"Model loading failed: {e}")


In [ ]:
from bertopic import BERTopic

OUT_DIR = "/content/drive/MyDrive/wired_project"

topic_model = BERTopic.load(f"{OUT_DIR}/bertopic_final_model_v4_stopwords")
print("Model loaded successfully")

sim_fig = topic_model.visualize_heatmap()
sim_fig.write_html(f"{OUT_DIR}/bertopic_similarity_heatmap_v4.html")
print("Similarity heatmap saved as HTML")

hier_fig = topic_model.visualize_hierarchy()
hier_fig.write_html(f"{OUT_DIR}/bertopic_hierarchy_v4.html")
print("Hierarchy visualization saved as HTML")

In [ ]:
# Method 1: Sentence embedding similarity (MiniLM + seed phrases)
from sentence_transformers import SentenceTransformer, util
import numpy as np
import pandas as pd

# Concatenate keywords into topic sentences
topic_ids = topic_info[topic_info['Topic'] != -1]['Topic'].tolist()
topic_texts = []
for topic_id in topic_ids:
    words = topic_model.get_topic(topic_id)
    topic_texts.append(" ".join([w[0] for w in words]))

# Define seed phrases
seed_phrases = [
    "concerns about risk", "technological threat", "danger of emerging technology", "growing public concern",
    "threats from AI", "risks of data privacy", "cybersecurity failures", "dangers of automation",
    "surveillance society", "algorithmic bias", "loss of control", "lack of effective regulation",
    "government surveillance", "corporate data misuse", "platform manipulation", "unfair algorithms",
    "concerns about safety", "digital security risks", "freedom under threat", "protection of user rights"
]

# Load MiniLM model and encode
sim_model = SentenceTransformer("all-MiniLM-L6-v2")
topic_embeddings = sim_model.encode(topic_texts, convert_to_tensor=True)
seed_embeddings = sim_model.encode(seed_phrases, convert_to_tensor=True)

# Compute maximum similarity
similarities = util.cos_sim(topic_embeddings, seed_embeddings).cpu().numpy()
max_sim_scores = np.max(similarities, axis=1)

# Save results
topic_info_embed = topic_info[topic_info["Topic"] != -1].copy()
topic_info_embed["embedding_similarity"] = max_sim_scores

# Filter topics
THRESHOLD = 0.2
filtered_embed = topic_info_embed[topic_info_embed["embedding_similarity"] > THRESHOLD]
print(f"Embedding-based method selected {len(filtered_embed)} topics")

In [ ]:
# Method 2: Zero-shot classification (BART MNLI + same seed phrases)
from transformers import pipeline
from tqdm import tqdm

# Reuse topic_texts and seed_phrases
zsc = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

zero_shot_scores = []
for text in tqdm(topic_texts, desc="Zero-shot scoring"):
    result = zsc(text, seed_phrases)
    max_score = max(result["scores"])
    zero_shot_scores.append(max_score)

# Save results
topic_info_zsc = topic_info[topic_info["Topic"] != -1].copy()
topic_info_zsc["zero_shot_score"] = zero_shot_scores

# Filter topics
THRESHOLD = 0.2
filtered_zsc = topic_info_zsc[topic_info_zsc["zero_shot_score"] > THRESHOLD]
print(f"Zero-shot method selected {len(filtered_zsc)} topics")


In [ ]:
# Extract Topic IDs
embed_ids = set(filtered_embed["Topic"])
zsc_ids = set(filtered_zsc["Topic"])

# Compute intersection and union
intersection_ids = embed_ids & zsc_ids
union_ids = embed_ids | zsc_ids

# Print results
print("Intersection topic count:", len(intersection_ids))
print("Intersection Topic IDs:", sorted(intersection_ids))

print("\nUnion topic count:", len(union_ids))
print("Union Topic IDs:", sorted(union_ids))

# Visualize distribution
import matplotlib.pyplot as plt

venn_labels = ["Embedding only", "Zero-shot only", "Both"]
plt.figure(figsize=(6, 4))
plt.title("Intersection vs Union")
plt.bar(venn_labels, [len(embed_ids - zsc_ids), len(zsc_ids - embed_ids), len(intersection_ids)])
plt.ylabel("Number of Topics")
plt.show()

In [ ]:
# Get list of intersection Topic IDs
intersection_ids = list(embed_ids & zsc_ids)

# Print keywords for each intersected topic
for tid in sorted(intersection_ids):
    keywords = topic_model.get_topic(tid)
    keywords_str = ", ".join([kw[0] for kw in keywords[:10]])
    print(f"Topic {tid}: {keywords_str}")

In [ ]:
# Re-cluster topics

from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import pandas as pd
import matplotlib.pyplot as plt

OUT_DIR = "/content/drive/MyDrive/wired_project"
topic_model = BERTopic.load(f"{OUT_DIR}/bertopic_final_model_v4_stopwords")

target_topic_ids = [
    0, 1, 4, 5, 7, 8, 12, 13, 14, 16, 17, 18, 21,
    23, 24, 26, 28, 31, 32, 33, 36, 42, 44, 49, 52,
    55, 56, 58, 59, 61, 62, 63, 64
]

# Extract keywords text for each topic
topic_texts = []
for tid in target_topic_ids:
    kws = topic_model.get_topic(tid)
    topic_texts.append(" ".join([w for w, _ in kws[:10]]))

# Sentence embeddings
embed_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = embed_model.encode(topic_texts, show_progress_bar=False)

# KMeans clustering (K=5)
n_clusters = 5
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init="auto")
labels = kmeans.fit_predict(embeddings)

# Result table
cluster_df = pd.DataFrame({
    "Topic_ID": target_topic_ids,
    "Keywords": topic_texts,
    "Cluster": labels
})

# Print topics per cluster
for i in range(n_clusters):
    subset = cluster_df[cluster_df["Cluster"] == i]
    print(f"\nCluster {i} ({len(subset)} topics)")
    for _, r in subset.iterrows():
        print(f"  - Topic {r['Topic_ID']}: {r['Keywords']}")

# PCA 2D visualization
pca = PCA(n_components=2, random_state=42)
xy = pca.fit_transform(embeddings)
plt.figure(figsize=(8, 6))
for i in range(n_clusters):
    pts = xy[labels == i]
    plt.scatter(pts[:, 0], pts[:, 1], label=f"Cluster {i}")
plt.title("Topic-level clustering (K=5)")
plt.legend()
plt.show()

# Save Topic→Cluster mapping
topic_to_cluster = dict(zip(cluster_df["Topic_ID"], cluster_df["Cluster"]))


In [ ]:
import pandas as pd
import numpy as np
from umap import UMAP
import matplotlib.pyplot as plt

CSV_PATH = "/content/drive/MyDrive/wired_project/wired_2014_2024_cleaned.csv"
EMBED_PATH = "/content/drive/MyDrive/wired_project/wired_bge-base-v1.5_len512.memmap"
DIM = 768

df = pd.read_csv(CSV_PATH)
docs_all = df["text"].dropna().astype(str).tolist()
emb_all = np.memmap(EMBED_PATH, dtype=np.float32, mode="r", shape=(len(docs_all), DIM))

# Get topic assignment for each document
doc_info = topic_model.get_document_info(docs_all)

# Keep only target topic documents
mask = doc_info["Topic"].isin(target_topic_ids)
doc_subset = doc_info[mask].copy()

# Map topics to cluster IDs
doc_subset["Cluster"] = doc_subset["Topic"].map(topic_to_cluster)

# Map cluster IDs to names
cluster_names = {
    0: "Mobility and Digital Infrastructure",
    1: "Surveillance, Privacy and Digital Governance",
    2: "Emerging Tech and Entertainment",
    3: "Digital Lifestyle and Consumer Tech",
    4: "Technology Policy and Communication Systems"
}
doc_subset["Cluster_Name"] = doc_subset["Cluster"].map(cluster_names)

# UMAP reduction
umap_model = UMAP(n_neighbors=15, min_dist=0.1, n_components=2, metric="cosine", random_state=42)
doc_embeddings = emb_all[doc_subset.index]
xy = umap_model.fit_transform(doc_embeddings)

# Plot
plt.figure(figsize=(12, 8))
for cid, cname in cluster_names.items():
    mask = doc_subset["Cluster"] == cid
    plt.scatter(
        xy[mask, 0], xy[mask, 1],
        s=3, alpha=0.6,
        label=cname
    )

plt.title("UMAP of Wired Articles in Tech-Risk Topics")
plt.xlabel("UMAP-1")
plt.ylabel("UMAP-2")
plt.legend(title="Cluster", fontsize=9)
plt.tight_layout()
plt.savefig("/content/drive/MyDrive/wired_project/cluster_umap_named.png", dpi=300)
plt.show()


In [ ]:
from bertopic import BERTopic
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

topic_model = BERTopic.load(f"{OUT_DIR}/bertopic_final_model_v4_stopwords")

df = pd.read_csv(CSV_PATH)
docs = df["text"].dropna().astype(str).tolist()
timestamps = df["date"].tolist()

print("Model and data loaded")
print(f"Number of documents: {len(docs)}")

def analyze_cluster_1_basic():
    """Basic analysis of Cluster 1: Surveillance, Privacy and Digital Governance"""

    print("Cluster 1 Analysis: Surveillance, Privacy and Digital Governance")
    print("=" * 70)

    cluster_1_topics = [7, 12, 23, 26, 28, 32, 33, 36, 52, 55, 62, 63]

    cluster_docs = []
    for i, topic_id in enumerate(topic_model.topics_):
        if topic_id in cluster_1_topics:
            cluster_docs.append({
                'doc_id': i,
                'text': docs[i],
                'timestamp': pd.to_datetime(timestamps[i]),
                'topic_id': topic_id,
                'year': pd.to_datetime(timestamps[i]).year,
                'month': pd.to_datetime(timestamps[i]).month
            })

    cluster_df = pd.DataFrame(cluster_docs)

    print(f"Total documents: {len(cluster_df)}")
    print(f"Number of topics: {len(cluster_1_topics)}")
    print(f"Time span: {cluster_df['year'].min()} - {cluster_df['year'].max()}")

    print(f"\nTOPIC BREAKDOWN:")
    topic_analysis = {}
    for topic_id in cluster_1_topics:
        topic_docs = cluster_df[cluster_df['topic_id'] == topic_id]
        topic_keywords = topic_model.get_topic(topic_id)

        topic_analysis[topic_id] = {
            'doc_count': len(topic_docs),
            'keywords': [word for word, score in topic_keywords[:5]],
            'peak_year': topic_docs.groupby('year').size().idxmax() if len(topic_docs) > 0 else None,
            'avg_doc_length': np.mean([len(doc.split()) for doc in topic_docs['text']]) if len(topic_docs) > 0 else 0
        }

        keywords_str = ', '.join(topic_analysis[topic_id]['keywords'])
        print(f"Topic {topic_id}: {len(topic_docs)} docs - {keywords_str}")
        if topic_analysis[topic_id]['peak_year']:
            print(f"  Peak year: {topic_analysis[topic_id]['peak_year']}")

    yearly_counts = cluster_df.groupby('year').size().reset_index(name='count')
    peak_years = yearly_counts.nlargest(3, 'count')

    print(f"\nTEMPORAL PATTERNS:")
    print("Peak years:")
    for _, row in peak_years.iterrows():
        print(f"  {row['year']}: {row['count']} articles")

    key_events = identify_key_events_cluster_1(yearly_counts)

    create_cluster_1_visualizations(cluster_df, yearly_counts, topic_analysis)

    return {
        'cluster_df': cluster_df,
        'topic_analysis': topic_analysis,
        'yearly_counts': yearly_counts,
        'key_events': key_events
    }

def identify_key_events_cluster_1(yearly_counts):
    """Identify key events that may explain temporal patterns"""

    key_events = {
        2013: "Edward Snowden NSA revelations",
        2016: "Cambridge Analytica data collection begins",
        2018: "Cambridge Analytica scandal breaks, GDPR implemented",
        2019: "Facial recognition controversies intensify",
        2020: "COVID-19 surveillance concerns, contact tracing debates"
    }

    events_analysis = []
    for year, event in key_events.items():
        if year in yearly_counts['year'].values:
            count = yearly_counts[yearly_counts['year'] == year]['count'].iloc[0]
            events_analysis.append({
                'year': year,
                'event': event,
                'article_count': count
            })

    print(f"\nKEY EVENTS CORRELATION:")
    for event in events_analysis:
        print(f"  {event['year']}: {event['event']} - {event['article_count']} articles")

    return events_analysis

def create_cluster_1_visualizations(cluster_df, yearly_counts, topic_analysis):
    """Create comprehensive visualizations for Cluster 1"""

    fig, axes = plt.subplots(2, 3, figsize=(18, 12))

    # Temporal trends with key events
    axes[0, 0].plot(yearly_counts['year'], yearly_counts['count'],
                    marker='o', linewidth=2, markersize=6, color='darkred')
    key_event_years = [2013, 2018]
    for year in key_event_years:
        if year in yearly_counts['year'].values:
            count = yearly_counts[yearly_counts['year'] == year]['count'].iloc[0]
            axes[0, 0].annotate(f'{year}\nKey Event',
                               xy=(year, count), xytext=(year, count+5),
                               arrowprops=dict(arrowstyle='->', color='red'),
                               ha='center', fontsize=8)
    axes[0, 0].set_title('Cluster 1: Temporal Trends with Key Events')
    axes[0, 0].set_xlabel('Year')
    axes[0, 0].set_ylabel('Number of Articles')
    axes[0, 0].grid(True, alpha=0.3)

    # Topic distribution (top 8)
    topic_counts = {tid: analysis['doc_count'] for tid, analysis in topic_analysis.items()}
    sorted_topics = sorted(topic_counts.items(), key=lambda x: x[1], reverse=True)
    topic_ids, counts = zip(*sorted_topics[:8])
    axes[0, 1].bar(range(len(topic_ids)), counts, color='steelblue')
    axes[0, 1].set_title('Topic Distribution (Top 8)')
    axes[0, 1].set_xlabel('Topic ID')
    axes[0, 1].set_ylabel('Document Count')
    axes[0, 1].set_xticks(range(len(topic_ids)))
    axes[0, 1].set_xticklabels(topic_ids, rotation=45)

    # Monthly trends in peak years
    peak_years_list = [2013, 2018, 2019]
    monthly_data = cluster_df[cluster_df['year'].isin(peak_years_list)]
    monthly_counts = monthly_data.groupby(['year', 'month']).size().reset_index(name='count')
    for year in peak_years_list:
        year_data = monthly_counts[monthly_counts['year'] == year]
        if len(year_data) > 0:
            axes[0, 2].plot(year_data['month'], year_data['count'],
                           marker='o', label=f'{year}', linewidth=2)
    axes[0, 2].set_title('Monthly Trends (Peak Years)')
    axes[0, 2].set_xlabel('Month')
    axes[0, 2].set_ylabel('Articles')
    axes[0, 2].legend()
    axes[0, 2].grid(True, alpha=0.3)

    # Topic-year heatmap
    topic_year = cluster_df.groupby(['year', 'topic_id']).size().unstack(fill_value=0)
    top_5_topics = [tid for tid, _ in sorted_topics[:5]]
    sns.heatmap(topic_year[top_5_topics].T, annot=True, fmt='d',
                ax=axes[1, 0], cmap='Reds', cbar_kws={'label': 'Article Count'})
    axes[1, 0].set_title('Topic Activity Heatmap (Top 5 Topics)')
    axes[1, 0].set_xlabel('Year')
    axes[1, 0].set_ylabel('Topic ID')

    # Document length distribution
    doc_lengths = [len(doc.split()) for doc in cluster_df['text']]
    axes[1, 1].hist(doc_lengths, bins=25, alpha=0.7, color='lightgreen', edgecolor='black')
    axes[1, 1].axvline(np.mean(doc_lengths), color='red', linestyle='--',
                       label=f'Mean: {np.mean(doc_lengths):.0f} words')
    axes[1, 1].set_title('Document Length Distribution')
    axes[1, 1].set_xlabel('Word Count')
    axes[1, 1].set_ylabel('Frequency')
    axes[1, 1].legend()

    # Frequent keywords
    all_keywords = []
    for topic_id, analysis in topic_analysis.items():
        for keyword in analysis['keywords']:
            all_keywords.extend([keyword] * analysis['doc_count'])
    keyword_counts = Counter(all_keywords)
    top_keywords = keyword_counts.most_common(10)
    keywords, counts = zip(*top_keywords)
    axes[1, 2].barh(range(len(keywords)), counts, color='orange')
    axes[1, 2].set_title('Most Frequent Keywords')
    axes[1, 2].set_xlabel('Frequency')
    axes[1, 2].set_yticks(range(len(keywords)))
    axes[1, 2].set_yticklabels(keywords)

    plt.tight_layout()
    plt.show()

print("Starting analysis of Cluster 1...")
cluster_1_results = analyze_cluster_1_basic()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bertopic import BERTopic

OUT_DIR = "/content/drive/MyDrive/wired_project"
CSV_PATH = "/content/drive/MyDrive/wired_project/wired_2014_2024_cleaned.csv"

topic_model = BERTopic.load(f"{OUT_DIR}/bertopic_final_model_v4_stopwords")
df = pd.read_csv(CSV_PATH)
docs = df["text"].dropna().astype(str).tolist()
timestamps = df["date"].tolist()
doc_info = topic_model.get_document_info(docs)

print("Model and data reloaded")
print(f"Total documents: {len(docs)}")
print(f"Number of topics: {len(topic_model.get_topics())}")

def create_cluster_1_clean_visualizations():
    """Generate clean visualizations for Cluster 1"""

    cluster_1_topics = [7, 12, 23, 26, 28, 32, 33, 36, 52, 55, 62, 63]
    cluster_1_df = doc_info[doc_info["Topic"].isin(cluster_1_topics)].copy()
    cluster_1_df["text"] = df.loc[cluster_1_df.index, "text"]
    cluster_1_df["year"] = pd.to_datetime(df.loc[cluster_1_df.index, "date"]).dt.year

    print(f"Cluster 1 documents: {len(cluster_1_df)}")

    fig, axes = plt.subplots(2, 2, figsize=(16, 10))

    # Yearly trend with key events
    yearly_counts = cluster_1_df.groupby('year').size()
    axes[0, 0].plot(yearly_counts.index, yearly_counts.values,
                   marker='o', linewidth=3, markersize=8, color='red')

    key_events = {
        2016: 'CA Data Collection',
        2018: 'CA Scandal & GDPR',
        2019: 'Facial Recognition',
        2020: 'COVID Surveillance'
    }

    for year, event in key_events.items():
        if year in yearly_counts.index:
            count = yearly_counts[year]
            axes[0, 0].annotate(event, xy=(year, count),
                               xytext=(year, count + 60),
                               arrowprops=dict(arrowstyle='->', color='blue', lw=2),
                               ha='center', fontsize=9,
                               bbox=dict(boxstyle="round,pad=0.3", facecolor="lightyellow"))

    axes[0, 0].set_title('Cluster 1: Surveillance Risk Coverage Timeline', fontweight='bold')
    axes[0, 0].set_xlabel('Year')
    axes[0, 0].set_ylabel('Number of Articles')
    axes[0, 0].grid(True, alpha=0.3)
    max_count = yearly_counts.max()
    axes[0, 0].set_ylim(0, max_count * 1.3)

    # Topic distribution
    topic_doc_counts = {}
    for topic_id in cluster_1_topics:
        topic_docs = cluster_1_df[cluster_1_df['Topic'] == topic_id]
        topic_doc_counts[topic_id] = len(topic_docs)

    sorted_topics = sorted(topic_doc_counts.items(), key=lambda x: x[1], reverse=True)[:8]
    topic_ids, doc_counts = zip(*sorted_topics)

    bars = axes[0, 1].bar(range(len(topic_ids)), doc_counts,
                         color='steelblue', alpha=0.8)
    axes[0, 1].set_title('Topic Distribution in Surveillance Cluster', fontweight='bold')
    axes[0, 1].set_xlabel('Topic ID')
    axes[0, 1].set_ylabel('Document Count')
    axes[0, 1].set_xticks(range(len(topic_ids)))
    axes[0, 1].set_xticklabels([f'T{tid}' for tid in topic_ids], rotation=45)

    for bar, count in zip(bars, doc_counts):
        axes[0, 1].text(bar.get_x() + bar.get_width()/2,
                       bar.get_height() + max(doc_counts)*0.02,
                       str(count), ha='center', va='bottom', fontweight='bold')

    # Evolution of top 5 topics
    top_5_topics = [tid for tid, _ in sorted_topics[:5]]
    colors = ['red', 'blue', 'green', 'orange', 'purple']

    for i, topic_id in enumerate(top_5_topics):
        topic_docs = cluster_1_df[cluster_1_df['Topic'] == topic_id]
        if len(topic_docs) > 0:
            topic_yearly = topic_docs.groupby('year').size()
            axes[1, 0].plot(topic_yearly.index, topic_yearly.values,
                           marker='o', label=f'Topic {topic_id}',
                           linewidth=2, color=colors[i])

    axes[1, 0].set_title('Top 5 Topics Evolution', fontweight='bold')
    axes[1, 0].set_xlabel('Year')
    axes[1, 0].set_ylabel('Articles per Topic')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)

    # Document length distribution
    cluster_1_df['word_count'] = cluster_1_df['text'].apply(lambda x: len(str(x).split()))
    axes[1, 1].hist(cluster_1_df['word_count'], bins=30,
                   color='lightgreen', alpha=0.7, edgecolor='black')
    mean_length = cluster_1_df['word_count'].mean()
    axes[1, 1].axvline(mean_length, color='red', linestyle='--', linewidth=2,
                      label=f'Mean: {mean_length:.0f} words')
    axes[1, 1].set_title('Article Length Distribution', fontweight='bold')
    axes[1, 1].set_xlabel('Word Count')
    axes[1, 1].set_ylabel('Frequency')
    axes[1, 1].legend()

    plt.tight_layout()
    plt.show()

    # Print statistics
    print("\nSTATISTICS FOR WRITING:")
    print("="*40)
    print("Top Topics:")
    for i, (topic_id, count) in enumerate(sorted_topics[:5], 1):
        percentage = (count / len(cluster_1_df)) * 100
        print(f"  {i}. Topic {topic_id}: {count} docs ({percentage:.1f}%)")

    print(f"\nTemporal Pattern:")
    print(f"  Peak year: {yearly_counts.idxmax()} ({yearly_counts.max()} articles)")
    print(f"  Total span: {cluster_1_df['year'].min()}-{cluster_1_df['year'].max()}")

create_cluster_1_clean_visualizations()


In [ ]:
# NER
!pip -q install spacy
!python -m spacy download en_core_web_trf

import pandas as pd
import numpy as np
import spacy
from spacy.pipeline import EntityRuler
from tqdm import tqdm
from collections import Counter
from itertools import combinations
import matplotlib.pyplot as plt
from bertopic import BERTopic

topic_model = BERTopic.load(f"{OUT_DIR}/bertopic_final_model_v4_stopwords")

df = pd.read_csv(CSV_PATH)
docs = df["text"].dropna().astype(str).tolist()

print("Data and model loaded, number of documents:", len(docs))

# Extract Cluster 1 documents
cluster_1_topics = [7, 12, 23, 26, 28, 32, 33, 36, 52, 55, 62, 63]
doc_info = topic_model.get_document_info(docs)
cluster_df = doc_info[doc_info["Topic"].isin(cluster_1_topics)].copy()
cluster_df["text"] = df.loc[cluster_df.index, "text"]
cluster_df["year"] = pd.to_datetime(df.loc[cluster_df.index, "date"]).dt.year

print("Cluster 1 documents:", len(cluster_df))
cluster_df.head(2)

# Configure spaCy pipeline (Transformer + EntityRuler)
nlp = spacy.load("en_core_web_trf", disable=["tagger", "parser", "lemmatizer"])
ruler = nlp.add_pipe("entity_ruler", before="ner")

patterns = [
    {"label": "LAW", "pattern": "GDPR"},
    {"label": "LAW", "pattern": "General Data Protection Regulation"},
    {"label": "ORG", "pattern": "Cambridge Analytica"},
    {"label": "ORG", "pattern": "NSA"},
    {"label": "ORG", "pattern": "Wikileaks"},
    {"label": "ORG", "pattern": "Clearview AI"},
    {"label": "ORG", "pattern": "NSO Group"},
    {"label": "PRODUCT", "pattern": "Pegasus"},
    {"label": "ORG", "pattern": "Meta"},
    {"label": "ORG", "pattern": "Facebook"},
    {"label": "ORG", "pattern": "WhatsApp"},
    {"label": "ORG", "pattern": "TikTok"},
    {"label": "ORG", "pattern": "EU"},
    {"label": "GPE", "pattern": "European Union"},
    {"label": "PERSON", "pattern": "Edward Snowden"},
    {"label": "PERSON", "pattern": "Julian Assange"},
    {"label": "PERSON", "pattern": "Mark Zuckerberg"},
    {"label": "TECH", "pattern": "end-to-end encryption"},
    {"label": "TECH", "pattern": "facial recognition"},
    {"label": "TECH", "pattern": "contact tracing"},
    {"label": "TECH", "pattern": "net neutrality"},
]
ruler.add_patterns(patterns)

alias_map = {
    "EU": "European Union",
    "European Union": "European Union",
    "UK": "United Kingdom",
    "U.K.": "United Kingdom",
    "US": "United States",
    "U.S.": "United States",
    "Meta": "Facebook",
    "Facebook": "Facebook",
    "General Data Protection Regulation": "GDPR",
}

def canonical(name: str) -> str:
    return alias_map.get(name, name)

# Run NER
rows = []
for doc, (_, row) in tqdm(
    zip(nlp.pipe(cluster_df["text"].tolist(), batch_size=16), cluster_df.iterrows()),
    total=len(cluster_df)
):
    y = int(row["year"])
    ents = [(canonical(ent.text), ent.label_) for ent in doc.ents]
    ents = [(t, l) for (t, l) in ents if l in {"PERSON", "ORG", "GPE", "LAW", "PRODUCT", "NORP", "TECH"}]
    if ents:
        rows.append({"year": y, "ents": ents})

ner_df = pd.DataFrame(rows)
print("NER completed, processed documents:", len(ner_df))

# Frequent entity statistics
counter_all = Counter()
for ents in ner_df["ents"]:
    counter_all.update([e[0] for e in ents])

top_entities = counter_all.most_common(20)
print("Top 20 entities:")
print(pd.DataFrame(top_entities, columns=["entity", "count"]))

# Temporal trends
def entity_year_counts(targets=None, top_k=15):
    df_list = []
    for year, ents in zip(ner_df["year"], ner_df["ents"]):
        for t, l in ents:
            df_list.append((year, t, l))
    edf = pd.DataFrame(df_list, columns=["year", "entity", "label"])
    if targets is None:
        targets = [e for e, _ in counter_all.most_common(top_k)]
    sub = edf[edf["entity"].isin(targets)]
    pivot = sub.groupby(["year", "entity"]).size().unstack(fill_value=0).sort_index()
    return pivot

trend = entity_year_counts(targets=["Facebook", "Cambridge Analytica", "GDPR", "Edward Snowden", "NSA", "facial recognition"])
trend.plot(figsize=(10, 6), marker="o", title="Cluster 1 - Entity Temporal Trends")
plt.show()

# Co-occurrence network
co_counter = Counter()
for ents in ner_df["ents"]:
    uniq = sorted(set([e[0] for e in ents]))
    for a, b in combinations(uniq, 2):
        co_counter[(a, b)] += 1

edges = pd.DataFrame([(a, b, w) for (a, b), w in co_counter.items()],
                     columns=["source", "target", "weight"])
print("Top 20 co-occurrence relations:")
print(edges.sort_values("weight", ascending=False).head(20))


In [ ]:
# Cluster 1 spaCy analysis

import pandas as pd
import numpy as np
import spacy
from tqdm import tqdm
from collections import Counter, defaultdict
from bertopic import BERTopic

OUT_DIR = "/content/drive/MyDrive/wired_project"
CSV_PATH = "/content/drive/MyDrive/wired_project/wired_2014_2024_cleaned.csv"

topic_model = BERTopic.load(f"{OUT_DIR}/bertopic_final_model_v4_stopwords")
df = pd.read_csv(CSV_PATH)
docs = df["text"].dropna().astype(str).tolist()

print("Model and data reloaded, number of documents:", len(docs))

cluster_1_topics = [7, 12, 23, 26, 28, 32, 33, 36, 52, 55, 62, 63]
doc_info = topic_model.get_document_info(docs)
cluster_df = doc_info[doc_info["Topic"].isin(cluster_1_topics)].copy()
cluster_df["text"] = df.loc[cluster_df.index, "text"]
cluster_df["year"] = pd.to_datetime(df.loc[cluster_df.index, "date"]).dt.year

print("Cluster 1 documents:", len(cluster_df))

def improved_spacy_analysis(cluster_df, sample_size=1000):
    """Improved spaCy analysis"""

    nlp = spacy.load("en_core_web_sm")

    def filter_entity(ent):
        text = ent.text.strip().lower()
        if len(text) < 2:
            return False
        if text in ['us', 'u.s.']:
            return False
        if 'wired' in text:
            return False
        if text.isdigit():
            return False
        if text in ['new', 'first', 'last', 'good', 'best', 'old', 'big', 'small']:
            return False
        return True

    def filter_cooccurrence(ent1, ent2):
        if 'wired' in ent1.lower() or 'wired' in ent2.lower():
            return False
        if ent1.lower() == ent2.lower():
            return False
        return True

    print(f"Starting improved spaCy analysis, sample size: {min(sample_size, len(cluster_df))}")

    if sample_size < len(cluster_df):
        sample_docs = cluster_df.sample(n=sample_size, random_state=42)
    else:
        sample_docs = cluster_df

    entity_counts = Counter()
    cooccurrence_counts = defaultdict(int)
    processed = 0

    for idx, row in tqdm(sample_docs.iterrows(), total=len(sample_docs)):
        try:
            doc = nlp(row['text'][:3000])

            filtered_entities = []
            for ent in doc.ents:
                if ent.label_ in ['PERSON', 'ORG', 'GPE'] and filter_entity(ent):
                    entity_text = ent.text.strip()
                    entity_counts[entity_text] += 1
                    filtered_entities.append(entity_text)

            if 1 < len(filtered_entities) < 20:
                for i, ent1 in enumerate(filtered_entities):
                    for ent2 in filtered_entities[i+1:]:
                        if filter_cooccurrence(ent1, ent2):
                            pair = tuple(sorted([ent1, ent2]))
                            cooccurrence_counts[pair] += 1

            processed += 1

        except Exception:
            continue

    print(f"Processing completed, documents processed: {processed}")
    return entity_counts, cooccurrence_counts

entity_counts, cooccurrence_counts = improved_spacy_analysis(cluster_df, sample_size=1000)

print("\n=== Improved Entity Recognition Results ===")
print("Top 20 Entities:")
for i, (entity, count) in enumerate(entity_counts.most_common(20)):
    print(f"{i+1:2d}. {entity:<20} {count:>4}")

print("\n=== Improved Co-occurrence Results ===")
print("Top 15 Entity Co-occurrences:")
for i, ((ent1, ent2), count) in enumerate(sorted(cooccurrence_counts.items(), key=lambda x: x[1], reverse=True)[:15]):
    print(f"{i+1:2d}. {ent1:<15} <-> {ent2:<15} {count:>3}")


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

def power_dynamics_timeline_analysis(cluster_df):
    """Analyze the changing prominence of power actors over time"""

    power_categories = {
        'Tech Giants': ['google', 'apple', 'amazon', 'microsoft', 'facebook'],
        'Government': ['fbi', 'nsa', 'congress', 'white house'],
        'Foreign Powers': ['russia', 'china', 'uk'],
        'Individual Actors': ['donald trump', 'clinton', 'snowden']
    }

    yearly_power_data = []
    for year in sorted(cluster_df['year'].unique()):
        year_docs = cluster_df[cluster_df['year'] == year]
        year_text = ' '.join(year_docs['text'].astype(str)).lower()

        year_data = {'year': year}
        for category, entities in power_categories.items():
            category_mentions = sum(year_text.count(entity.lower()) for entity in entities)
            year_data[category] = category_mentions

        yearly_power_data.append(year_data)

    df_power = pd.DataFrame(yearly_power_data)

    # Stacked area chart
    plt.figure(figsize=(12, 8))
    plt.stackplot(
        df_power['year'],
        df_power['Tech Giants'],
        df_power['Government'],
        df_power['Foreign Powers'],
        df_power['Individual Actors'],
        labels=['Tech Giants', 'Government', 'Foreign Powers', 'Individual Actors'],
        alpha=0.8
    )
    plt.title('Power Actor Prominence in Surveillance Discourse (2014-2024)')
    plt.xlabel('Year')
    plt.ylabel('Mention Frequency')
    plt.legend(loc='upper left')
    plt.grid(True, alpha=0.3)
    plt.show()

    print("Yearly power actor data:")
    print(df_power)

    return df_power

# Run analysis
df_power = power_dynamics_timeline_analysis(cluster_df)


In [ ]:
def emotional_authority_mapping(cluster_df, entity_counts):
    """Analyze emotional-authoritative discourse strategies of key entities"""

    # Fear lexicon (expanded)
    fear_words = [
        "fear", "scared", "scary", "terrified", "terrifying", "terrible",
        "horror", "horrific", "horrifying", "fright", "frightening",
        "panic", "panic-stricken", "panic attack", "phobia",
        "danger", "dangerous", "risk", "risky", "hazard", "hazardous",
        "threat", "threatening", "menace", "peril", "exposed", "vulnerable",
        "crisis", "catastrophe", "catastrophic", "disaster", "calamity",
        "apocalypse", "collapse", "devastating", "devastated", "tragedy",
        "alarm", "alarming", "fearful", "fearsome", "dread", "dreadful",
        "panic", "shock", "shocking", "nightmare", "chaos", "uncertainty",
        "doom", "insecure", "frightened", "scare", "menacing", "scaremongering"
    ]

    # Authority / evidence lexicon
    authority_words = [
        "official", "officials", "government", "administration",
        "authorities", "agency", "organization", "institution",
        "committee", "commission", "department", "ministry",
        "expert", "experts", "specialist", "specialists",
        "scientist", "scientists", "researcher", "researchers",
        "academics", "professor", "scholars", "analyst", "analysts",
        "research", "study", "studies", "study shows", "studies show",
        "data", "data reveals", "statistics", "findings", "report", "reports",
        "survey", "evidence", "confirmed", "verified", "according", "according to",
        "official data", "statement", "authoritative", "documentation"
    ]

    top_entities = [entity for entity, count in entity_counts.most_common(12)]
    entity_profiles = []

    print("Processing entities for emotional-authority mapping...")

    for entity in top_entities:
        entity_docs = cluster_df[
            cluster_df['text'].str.contains(entity, case=False, na=False)
        ]
        if len(entity_docs) == 0:
            continue

        combined_text = ' '.join(entity_docs['text'].head(50)).lower()
        fear_score = sum(combined_text.count(word) for word in fear_words)
        authority_score = sum(combined_text.count(word) for word in authority_words)

        doc_count = len(entity_docs)
        word_count = len(combined_text.split())

        fear_score_norm = (fear_score / word_count) * 1000 if word_count > 0 else 0
        authority_score_norm = (authority_score / word_count) * 1000 if word_count > 0 else 0

        entity_profiles.append({
            'entity': entity,
            'fear_score': fear_score_norm,
            'authority_score': authority_score_norm,
            'doc_count': doc_count,
            'word_count': word_count
        })

        print(f"  {entity}: {doc_count} docs, fear={fear_score_norm:.2f}, authority={authority_score_norm:.2f}")

    df_profiles = pd.DataFrame(entity_profiles)

    plt.figure(figsize=(12, 8))

    def get_entity_color(entity):
        entity_lower = entity.lower()
        if any(tech in entity_lower for tech in ['google', 'apple', 'amazon', 'microsoft', 'facebook']):
            return 'blue'
        elif any(gov in entity_lower for gov in ['fbi', 'nsa', 'congress', 'white house']):
            return 'red'
        elif any(foreign in entity_lower for foreign in ['russia', 'china', 'uk']):
            return 'green'
        else:
            return 'orange'

    colors = [get_entity_color(entity) for entity in df_profiles['entity']]
    sizes = [min(doc_count * 5, 500) for doc_count in df_profiles['doc_count']]

    plt.scatter(
        df_profiles['authority_score'],
        df_profiles['fear_score'],
        s=sizes,
        c=colors,
        alpha=0.7,
        edgecolors='black',
        linewidth=0.5
    )

    for _, row in df_profiles.iterrows():
        plt.annotate(row['entity'],
                     (row['authority_score'], row['fear_score']),
                     xytext=(5, 5),
                     textcoords='offset points',
                     fontsize=9,
                     ha='left')

    plt.xlabel('Authority Language (per 1000 words)')
    plt.ylabel('Fear Language (per 1000 words)')
    plt.title('Discourse Strategy Mapping: Authority vs Fear in Surveillance Narratives')
    plt.grid(True, alpha=0.3)

    legend_elements = [
        plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='blue', markersize=10, label='Tech Companies'),
        plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='red', markersize=10, label='Government'),
        plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='green', markersize=10, label='Foreign Powers'),
        plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='orange', markersize=10, label='Others')
    ]
    plt.legend(handles=legend_elements, loc='upper right')

    plt.tight_layout()
    plt.show()

    print("\nEntity Discourse Profile:")
    print(df_profiles.sort_values('fear_score', ascending=False))

    return df_profiles

# Run emotional-authority analysis
df_profiles = emotional_authority_mapping(cluster_df, entity_counts)

In [ ]:
def facebook_cambridge_analytica_deep_dive(cluster_df):
    """Deep dive analysis of the Facebook / Cambridge Analytica case"""

    # Keywords for filtering
    facebook_keywords = ['facebook', 'cambridge analytica', 'zuckerberg', 'fb', 'social media platform']
    facebook_docs = cluster_df[
        cluster_df['text'].str.contains('|'.join(facebook_keywords), case=False, na=False)
    ].copy()

    print(f"Total Facebook-related documents: {len(facebook_docs)}")

    # Yearly counts
    yearly_fb = facebook_docs.groupby('year').size().reset_index(name='count')

    # Time periods
    periods = {
        'Pre-scandal (2014-2017)': facebook_docs[facebook_docs['year'] <= 2017],
        'Scandal Peak (2018)': facebook_docs[facebook_docs['year'] == 2018],
        'Post-scandal (2019-2024)': facebook_docs[facebook_docs['year'] >= 2019]
    }

    # Risk, sentiment, and regulatory vocabularies
    tech_risk_words = [
        "privacy", "data breach", "data leak", "data theft", "hacking", "hackers",
        "cyberattack", "cybersecurity", "phishing", "ransomware", "malware",
        "virus", "spyware", "trojan", "breach", "exposed", "unauthorized access",
        "surveillance", "mass surveillance", "facial recognition", "biometric data",
        "tracking", "monitoring", "spying", "espionage", "censorship", "manipulation",
        "propaganda", "misinformation", "disinformation", "deepfake", "fake news",
        "algorithm", "bias", "algorithmic bias", "automation", "black box",
        "opacity", "fairness", "discrimination", "unethical AI", "job loss",
        "replacement", "autonomous", "lethal autonomous weapon", "AI ethics",
        "monopoly", "platform power", "gatekeeping", "data monopoly",
        "addiction", "overuse", "misuse", "platform abuse", "shadowban",
        "exploit", "exploitation", "unsafe", "dangerous technology"
    ]

    negative_words = [
        "scandal", "violation", "abuse", "concern", "threat", "dangerous",
        "crisis", "catastrophe", "collapse", "chaos", "corruption",
        "failure", "fraud", "harm", "harmful", "toxic", "controversy",
        "backlash", "boycott", "risk", "fear", "alarm", "panic",
        "deadly", "fatal", "devastating", "nightmare", "unprecedented"
    ]

    regulatory_words = [
        "regulation", "oversight", "accountability", "transparency", "congress",
        "parliament", "legislation", "lawmakers", "law", "bill", "act",
        "policy", "governance", "authority", "commission", "agency",
        "standards", "compliance", "ethics", "responsibility",
        "investigation", "report", "committee", "task force", "ministry",
        "federal", "supreme court", "executive order"
    ]

    # Create figures
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))

    # Time trend
    axes[0, 0].plot(yearly_fb['year'], yearly_fb['count'], marker='o', linewidth=2, color='blue')
    axes[0, 0].axvline(x=2018, color='red', linestyle='--', alpha=0.7, label='CA Scandal')
    axes[0, 0].set_title('Facebook Coverage Over Time')
    axes[0, 0].set_xlabel('Year')
    axes[0, 0].set_ylabel('Number of Articles')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)

    # Period-based keyword analysis
    period_keywords = {}
    risk_evolution = {}

    for period_name, period_docs in periods.items():
        if len(period_docs) == 0:
            continue

        combined_text = ' '.join(period_docs['text']).lower()

        risk_evolution[period_name] = sum(combined_text.count(word) for word in tech_risk_words)

        period_keywords[period_name] = {
            'negative_sentiment': sum(combined_text.count(word) for word in negative_words),
            'regulatory_response': sum(combined_text.count(word) for word in regulatory_words),
            'doc_count': len(period_docs)
        }

    # Tech risk vocabulary evolution
    risk_df = pd.DataFrame(risk_evolution, index=['mentions']).T
    risk_df.plot(kind='bar', ax=axes[0, 1], legend=False, color='orange')
    axes[0, 1].set_title('Tech Risk Language Evolution')
    axes[0, 1].set_xlabel('Time Period')
    axes[0, 1].set_ylabel('Mention Count')
    axes[0, 1].tick_params(axis='x', rotation=45)

    # Sentiment & regulatory discourse
    sentiment_data = pd.DataFrame(period_keywords).T
    sentiment_data[['negative_sentiment', 'regulatory_response']].plot(kind='bar', ax=axes[1, 0])
    axes[1, 0].set_title('Discourse Tone Evolution')
    axes[1, 0].set_xlabel('Time Period')
    axes[1, 0].set_ylabel('Word Count')
    axes[1, 0].tick_params(axis='x', rotation=45)

    # Regulatory intensity
    reg_intensity = [data['regulatory_response']/data['doc_count'] if data['doc_count'] > 0 else 0
                     for data in period_keywords.values()]
    period_names = list(period_keywords.keys())
    axes[1, 1].bar(period_names, reg_intensity, color=['lightblue', 'red', 'lightgreen'])
    axes[1, 1].set_title('Regulatory Language Intensity')
    axes[1, 1].set_xlabel('Time Period')
    axes[1, 1].set_ylabel('Regulatory Words per Document')
    axes[1, 1].tick_params(axis='x', rotation=45)

    plt.tight_layout()
    plt.show()

    # Summary
    print("\n" + "="*60)
    print("FACEBOOK / CAMBRIDGE ANALYTICA CASE STUDY SUMMARY")
    print("="*60)

    for period_name, data in period_keywords.items():
        neg_intensity = data['negative_sentiment']/data['doc_count'] if data['doc_count'] > 0 else 0
        reg_intensity = data['regulatory_response']/data['doc_count'] if data['doc_count'] > 0 else 0
        print(f"\n{period_name}:")
        print(f"  Documents: {data['doc_count']}")
        print(f"  Negative sentiment intensity: {neg_intensity:.2f}")
        print(f"  Regulatory response intensity: {reg_intensity:.2f}")

    return facebook_docs, period_keywords

# Run analysis
facebook_docs, fb_analysis = facebook_cambridge_analytica_deep_dive(cluster_df)


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from datetime import datetime
from bertopic import BERTopic

# Reload data and model
OUT_DIR = "/content/drive/MyDrive/wired_project"
CSV_PATH = "/content/drive/MyDrive/wired_project/wired_2014_2024_cleaned.csv"

topic_model = BERTopic.load(f"{OUT_DIR}/bertopic_final_model_v4_stopwords")
df = pd.read_csv(CSV_PATH)
docs = df["text"].dropna().astype(str).tolist()

print("Model and data reloaded, number of documents:", len(docs))

# Cluster 1 data
cluster_1_topics = [7, 12, 23, 26, 28, 32, 33, 36, 52, 55, 62, 63]
doc_info = topic_model.get_document_info(docs)
cluster_df = doc_info[doc_info["Topic"].isin(cluster_1_topics)].copy()
cluster_df["text"] = df.loc[cluster_df.index, "text"]
cluster_df["year"] = pd.to_datetime(df.loc[cluster_df.index, "date"]).dt.year

print("Cluster 1 data prepared, documents:", len(cluster_df))

def analyze_cluster_2_complete():
    """Complete analysis of Cluster 2: Emerging Tech and Entertainment"""

    print("Cluster 2 Analysis: Emerging Tech and Entertainment")
    print("=" * 70)

    cluster_2_topics = [0, 1, 8, 17, 24, 42, 44, 49]
    cluster_2_df = doc_info[doc_info["Topic"].isin(cluster_2_topics)].copy()
    cluster_2_df["text"] = df.loc[cluster_2_df.index, "text"]
    cluster_2_df["year"] = pd.to_datetime(df.loc[cluster_2_df.index, "date"]).dt.year

    print(f"Total documents: {len(cluster_2_df)}")
    print(f"Number of topics: {len(cluster_2_topics)}")
    print(f"Time span: {cluster_2_df['year'].min()} - {cluster_2_df['year'].max()}")

    print(f"\nTOPIC BREAKDOWN:")
    topic_analysis = {}

    topic_descriptions = {
        0: "COVID-19/Medical AI/Vaccines",
        1: "VR Gaming/Nintendo/Xbox",
        8: "Drones/Aviation Technology",
        17: "Star Wars/Entertainment Media",
        24: "Camera/Photography Technology",
        42: "AI Games/AlphaGo/DeepMind",
        44: "Microsoft/Windows/Operating Systems",
        49: "Pokemon Go/Mobile Gaming"
    }

    for topic_id in cluster_2_topics:
        topic_docs = cluster_2_df[cluster_2_df['Topic'] == topic_id]
        topic_keywords = topic_model.get_topic(topic_id)

        topic_analysis[topic_id] = {
            'doc_count': len(topic_docs),
            'keywords': [word for word, score in topic_keywords[:5]],
            'description': topic_descriptions.get(topic_id, "Unknown"),
            'peak_year': topic_docs.groupby('year').size().idxmax() if len(topic_docs) > 0 else None
        }

        keywords_str = ', '.join(topic_analysis[topic_id]['keywords'])
        print(f"Topic {topic_id}: {len(topic_docs)} docs - {topic_analysis[topic_id]['description']}")
        print(f"  Keywords: {keywords_str}")
        if topic_analysis[topic_id]['peak_year']:
            print(f"  Peak year: {topic_analysis[topic_id]['peak_year']}")

    yearly_counts = cluster_2_df.groupby('year').size().reset_index(name='count')
    peak_years = yearly_counts.nlargest(3, 'count')

    print(f"\nTEMPORAL PATTERNS:")
    print("Peak years:")
    for _, row in peak_years.iterrows():
        print(f"  {row['year']}: {row['count']} articles")

    print(f"\nCOMPARISON WITH CLUSTER 1:")
    cluster_1_total = len(cluster_df)
    cluster_2_total = len(cluster_2_df)
    print(f"Cluster 1 (Surveillance): {cluster_1_total} documents")
    print(f"Cluster 2 (Emerging Tech): {cluster_2_total} documents")
    print(f"Coverage ratio (C2/C1): {cluster_2_total/cluster_1_total:.2f}")

    create_cluster_2_visualizations(cluster_2_df, yearly_counts, topic_analysis)

    return cluster_2_df, topic_analysis

def create_cluster_2_visualizations(cluster_2_df, yearly_counts, topic_analysis):
    """Visualizations for Cluster 2"""

    fig, axes = plt.subplots(2, 2, figsize=(15, 10))

    # 1. Yearly trend
    axes[0, 0].plot(yearly_counts['year'], yearly_counts['count'],
                    marker='o', linewidth=2, markersize=6, color='green')

    tech_events = {2016: 'AlphaGo vs Lee', 2020: 'COVID-19'}
    for year, event in tech_events.items():
        if year in yearly_counts['year'].values:
            count = yearly_counts[yearly_counts['year'] == year]['count'].iloc[0]
            axes[0, 0].annotate(f'{event}',
                               xy=(year, count), xytext=(year, count+15),
                               arrowprops=dict(arrowstyle='->', color='red'),
                               ha='center', fontsize=8)

    axes[0, 0].set_title('Cluster 2: Emerging Tech Coverage Over Time')
    axes[0, 0].set_xlabel('Year')
    axes[0, 0].set_ylabel('Number of Articles')
    axes[0, 0].grid(True, alpha=0.3)

    # 2. Topic distribution
    topic_counts = {tid: analysis['doc_count'] for tid, analysis in topic_analysis.items()}
    sorted_topics = sorted(topic_counts.items(), key=lambda x: x[1], reverse=True)
    topic_ids, counts = zip(*sorted_topics)

    axes[0, 1].bar(range(len(topic_ids)), counts, color='lightgreen')
    axes[0, 1].set_title('Topic Distribution in Cluster 2')
    axes[0, 1].set_xlabel('Topic ID')
    axes[0, 1].set_ylabel('Document Count')
    axes[0, 1].set_xticks(range(len(topic_ids)))
    axes[0, 1].set_xticklabels([f'T{tid}' for tid in topic_ids], rotation=45)

    # 3. Tech category pie chart
    tech_categories = {
        'AI/ML': [0, 42],
        'Gaming/VR': [1, 49],
        'Hardware/Systems': [8, 24, 44],
        'Entertainment': [17]
    }

    category_counts = {}
    for category, topic_list in tech_categories.items():
        count = sum(topic_analysis[tid]['doc_count'] for tid in topic_list if tid in topic_analysis)
        category_counts[category] = count

    categories, counts = zip(*category_counts.items())
    colors = ['lightblue', 'lightcoral', 'lightgreen', 'gold']
    axes[1, 0].pie(counts, labels=categories, autopct='%1.1f%%',
                   startangle=90, colors=colors)
    axes[1, 0].set_title('Technology Category Distribution')

    # 4. Top 3 topics evolution
    top_3_topics = [tid for tid, _ in sorted_topics[:3]]
    for topic_id in top_3_topics:
        topic_docs = cluster_2_df[cluster_2_df['Topic'] == topic_id]
        if len(topic_docs) > 0:
            topic_yearly = topic_docs.groupby('year').size()
            axes[1, 1].plot(topic_yearly.index, topic_yearly.values,
                           marker='o', label=f'Topic {topic_id}', linewidth=2)

    axes[1, 1].set_title('Top 3 Topics Evolution')
    axes[1, 1].set_xlabel('Year')
    axes[1, 1].set_ylabel('Articles')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

def compare_cluster_1_vs_2_discourse(cluster_1_df, cluster_2_df):
    """Compare discourse features of Cluster 1 vs Cluster 2"""

    print("\nCOMPARATIVE DISCOURSE ANALYSIS: Surveillance vs Emerging Tech")
    print("=" * 70)

    surveillance_words = [
        "threat", "risk", "concern", "danger", "security", "privacy",
        "surveillance", "breach", "data breach", "data leak", "hack",
        "hacking", "cyberattack", "cybersecurity", "ransomware", "malware",
        "virus", "spyware", "trojan", "exposed", "unauthorized",
        "monitoring", "tracking", "spying", "espionage", "censorship",
        "manipulation", "propaganda", "misinformation", "disinformation",
        "deepfake", "fake news", "bias", "algorithmic bias", "black box",
        "opacity", "unethical", "unsafe", "harmful", "catastrophe",
        "collapse", "chaos", "corruption", "failure", "fraud", "toxic",
        "controversy", "backlash", "boycott", "fear", "panic", "alarm"
    ]

    innovation_words = [
        "breakthrough", "advancement", "innovation", "innovative", "progress",
        "future", "futuristic", "revolutionary", "cutting-edge", "novel",
        "development", "improvement", "evolution", "next generation",
        "transformative", "modernization", "discovery", "pioneering",
        "groundbreaking", "promising", "visionary", "trendsetting",
        "game-changing", "paradigm shift", "progressive", "emerging",
        "enhancement", "upgrading", "state-of-the-art"
    ]

    c1_sample = cluster_df.sample(n=min(200, len(cluster_df)), random_state=42)
    c2_sample = cluster_2_df.sample(n=min(200, len(cluster_2_df)), random_state=42)

    cluster_1_text = ' '.join(c1_sample['text'].astype(str)).lower()
    cluster_2_text = ' '.join(c2_sample['text'].astype(str)).lower()

    c1_risk_score = sum(cluster_1_text.count(word) for word in surveillance_words)
    c1_innovation_score = sum(cluster_1_text.count(word) for word in innovation_words)

    c2_risk_score = sum(cluster_2_text.count(word) for word in surveillance_words)
    c2_innovation_score = sum(cluster_2_text.count(word) for word in innovation_words)

    c1_words = len(cluster_1_text.split())
    c2_words = len(cluster_2_text.split())

    comparison_data = {
        'Cluster': ['Surveillance (C1)', 'Emerging Tech (C2)'],
        'Risk Language': [c1_risk_score/c1_words*1000 if c1_words > 0 else 0,
                         c2_risk_score/c2_words*1000 if c2_words > 0 else 0],
        'Innovation Language': [c1_innovation_score/c1_words*1000 if c1_words > 0 else 0,
                               c2_innovation_score/c2_words*1000 if c2_words > 0 else 0]
    }

    comparison_df = pd.DataFrame(comparison_data)

    x = range(len(comparison_df))
    width = 0.35

    plt.figure(figsize=(12, 6))
    plt.bar([i - width/2 for i in x], comparison_df['Risk Language'],
            width, label='Risk Language', color='red', alpha=0.7)
    plt.bar([i + width/2 for i in x], comparison_df['Innovation Language'],
            width, label='Innovation Language', color='blue', alpha=0.7)

    plt.xlabel('Coverage Type')
    plt.ylabel('Word Frequency (per 1000 words)')
    plt.title("Wired's Editorial Strategy: Risk vs Innovation Discourse")
    plt.xticks(x, comparison_df['Cluster'])
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

    print("Wired's Editorial Language Patterns:")
    for i, row in comparison_df.iterrows():
        print(f"{row['Cluster']}:")
        print(f"  Risk language intensity: {row['Risk Language']:.2f} per 1000 words")
        print(f"  Innovation language intensity: {row['Innovation Language']:.2f} per 1000 words")
        ratio = row['Innovation Language'] / row['Risk Language'] if row['Risk Language'] > 0 else float('inf')
        print(f"  Innovation/Risk ratio: {ratio:.2f}")

    return comparison_df

print("Starting full analysis of Cluster 2...")
cluster_2_df, cluster_2_analysis = analyze_cluster_2_complete()

print("\n comparing Cluster 1 vs Cluster 2...")
discourse_comparison = compare_cluster_1_vs_2_discourse(cluster_df, cluster_2_df)


In [ ]:
# Cluster 2 Analysis

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from datetime import datetime
from bertopic import BERTopic

# Reload data and model
OUT_DIR = "/content/drive/MyDrive/wired_project"
CSV_PATH = "/content/drive/MyDrive/wired_project/wired_2014_2024_cleaned.csv"

topic_model = BERTopic.load(f"{OUT_DIR}/bertopic_final_model_v4_stopwords")
df = pd.read_csv(CSV_PATH)
docs = df["text"].dropna().astype(str).tolist()

print("Model and data reloaded, number of documents:", len(docs))

# Cluster 1 data for comparison
cluster_1_topics = [7, 12, 23, 26, 28, 32, 33, 36, 52, 55, 62, 63]
doc_info = topic_model.get_document_info(docs)
cluster_df = doc_info[doc_info["Topic"].isin(cluster_1_topics)].copy()
cluster_df["text"] = df.loc[cluster_df.index, "text"]
cluster_df["year"] = pd.to_datetime(df.loc[cluster_df.index, "date"]).dt.year

print("Cluster 1 data prepared, documents:", len(cluster_df))

def analyze_cluster_2_complete():
    """Complete analysis of Cluster 2: Emerging Tech and Entertainment"""

    print("Cluster 2 Analysis: Emerging Tech and Entertainment")
    print("=" * 70)

    cluster_2_topics = [0, 1, 8, 17, 24, 42, 44, 49]
    cluster_2_df = doc_info[doc_info["Topic"].isin(cluster_2_topics)].copy()
    cluster_2_df["text"] = df.loc[cluster_2_df.index, "text"]
    cluster_2_df["year"] = pd.to_datetime(df.loc[cluster_2_df.index, "date"]).dt.year

    print(f"Total documents: {len(cluster_2_df)}")
    print(f"Number of topics: {len(cluster_2_topics)}")
    print(f"Time span: {cluster_2_df['year'].min()} - {cluster_2_df['year'].max()}")

    print(f"\nTOPIC BREAKDOWN:")
    topic_analysis = {}

    topic_descriptions = {
        0: "General Tech/Mixed Content",
        1: "VR Gaming/Nintendo/Xbox",
        8: "Drones/Aviation Technology",
        17: "Star Wars/Entertainment Media",
        24: "Camera/Photography Technology",
        42: "AI Games/AlphaGo/DeepMind",
        44: "Microsoft/Windows/Operating Systems",
        49: "Pokemon Go/Mobile Gaming"
    }

    for topic_id in cluster_2_topics:
        topic_docs = cluster_2_df[cluster_2_df['Topic'] == topic_id]
        topic_keywords = topic_model.get_topic(topic_id)

        topic_analysis[topic_id] = {
            'doc_count': len(topic_docs),
            'keywords': [word for word, score in topic_keywords[:5]],
            'description': topic_descriptions.get(topic_id, "Unknown"),
            'peak_year': topic_docs.groupby('year').size().idxmax() if len(topic_docs) > 0 else None
        }

        keywords_str = ', '.join(topic_analysis[topic_id]['keywords'])
        print(f"Topic {topic_id}: {len(topic_docs)} docs - {topic_analysis[topic_id]['description']}")
        print(f"  Keywords: {keywords_str}")
        if topic_analysis[topic_id]['peak_year']:
            print(f"  Peak year: {topic_analysis[topic_id]['peak_year']}")

    yearly_counts = cluster_2_df.groupby('year').size().reset_index(name='count')
    peak_years = yearly_counts.nlargest(3, 'count')

    print(f"\nTEMPORAL PATTERNS:")
    print("Peak years:")
    for _, row in peak_years.iterrows():
        print(f"  {row['year']}: {row['count']} articles")

    print(f"\nCOMPARISON WITH CLUSTER 1:")
    cluster_1_total = len(cluster_df)
    cluster_2_total = len(cluster_2_df)
    print(f"Cluster 1 (Surveillance): {cluster_1_total} documents")
    print(f"Cluster 2 (Emerging Tech): {cluster_2_total} documents")
    print(f"Coverage ratio (C2/C1): {cluster_2_total/cluster_1_total:.2f}")

    create_cluster_2_visualizations(cluster_2_df, yearly_counts, topic_analysis)

    return cluster_2_df, topic_analysis

def create_cluster_2_visualizations(cluster_2_df, yearly_counts, topic_analysis):
    """Visualizations for Cluster 2"""

    fig, axes = plt.subplots(2, 2, figsize=(15, 10))

    # 1. Yearly trend
    axes[0, 0].plot(yearly_counts['year'], yearly_counts['count'],
                    marker='o', linewidth=2, markersize=6, color='green')

    tech_events = {2016: 'AlphaGo vs Lee', 2020: 'COVID-19'}
    for year, event in tech_events.items():
        if year in yearly_counts['year'].values:
            count = yearly_counts[yearly_counts['year'] == year]['count'].iloc[0]
            axes[0, 0].annotate(f'{event}',
                               xy=(year, count), xytext=(year, count+15),
                               arrowprops=dict(arrowstyle='->', color='red'),
                               ha='center', fontsize=8)

    axes[0, 0].set_title('Cluster 2: Emerging Tech Coverage Over Time')
    axes[0, 0].set_xlabel('Year')
    axes[0, 0].set_ylabel('Number of Articles')
    axes[0, 0].grid(True, alpha=0.3)

    # 2. Topic distribution (excluding Topic 0)
    filtered_topics = {tid: analysis for tid, analysis in topic_analysis.items() if tid != 0}
    topic_counts = {tid: analysis['doc_count'] for tid, analysis in filtered_topics.items()}
    sorted_topics = sorted(topic_counts.items(), key=lambda x: x[1], reverse=True)
    topic_ids, counts = zip(*sorted_topics)

    axes[0, 1].bar(range(len(topic_ids)), counts, color='lightgreen')
    axes[0, 1].set_title('Topic Distribution (Focused Technologies)')
    axes[0, 1].set_xlabel('Topic ID')
    axes[0, 1].set_ylabel('Document Count')
    axes[0, 1].set_xticks(range(len(topic_ids)))
    axes[0, 1].set_xticklabels([f'T{tid}' for tid in topic_ids], rotation=45)

    # 3. Technology category pie chart (excluding Topic 0)
    tech_categories = {
        'Gaming/VR': [1, 49],
        'Hardware/Systems': [8, 24, 44],
        'Entertainment': [17],
        'AI/ML': [42]
    }

    category_counts = {}
    for category, topic_list in tech_categories.items():
        count = sum(topic_analysis[tid]['doc_count'] for tid in topic_list if tid in topic_analysis)
        category_counts[category] = count

    categories, counts = zip(*category_counts.items())
    colors = ['lightblue', 'orange', 'lightgreen', 'red']
    axes[1, 0].pie(counts, labels=categories, autopct='%1.1f%%',
                   startangle=90, colors=colors)
    axes[1, 0].set_title('Technology Category Distribution\n(Excluding General Content)')

    # 4. Top 3 topics evolution (excluding Topic 0)
    top_3_topics = [tid for tid, _ in sorted_topics[:3]]
    for topic_id in top_3_topics:
        topic_docs = cluster_2_df[cluster_2_df['Topic'] == topic_id]
        if len(topic_docs) > 0:
            topic_yearly = topic_docs.groupby('year').size()
            axes[1, 1].plot(topic_yearly.index, topic_yearly.values,
                           marker='o', label=f'Topic {topic_id}', linewidth=2)

    axes[1, 1].set_title('Top Focused Topics Evolution')
    axes[1, 1].set_xlabel('Year')
    axes[1, 1].set_ylabel('Articles')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    print(f"\nFocused Technology Distribution:")
    total_focused = sum(counts)
    for category, count in zip(categories, counts):
        print(f"  {category}: {count} docs ({count/total_focused*100:.1f}%)")

def compare_cluster_1_vs_2_discourse(cluster_1_df, cluster_2_df):
    """Compare discourse features of Cluster 1 vs Cluster 2"""

    print("\nCOMPARATIVE DISCOURSE ANALYSIS: Surveillance vs Emerging Tech")
    print("=" * 70)

    surveillance_words = [
        "threat", "risk", "concern", "danger", "security", "privacy",
        "surveillance", "breach", "data breach", "data leak", "hack",
        "hacking", "cyberattack", "cybersecurity", "ransomware", "malware",
        "virus", "spyware", "trojan", "exposed", "unauthorized",
        "monitoring", "tracking", "spying", "espionage", "censorship",
        "manipulation", "propaganda", "misinformation", "disinformation",
        "deepfake", "fake news", "bias", "algorithmic bias", "black box",
        "opacity", "unethical", "unsafe", "harmful", "catastrophe",
        "collapse", "chaos", "corruption", "failure", "fraud", "toxic",
        "controversy", "backlash", "boycott", "fear", "panic", "alarm"
    ]

    innovation_words = [
        "breakthrough", "advancement", "innovation", "innovative", "progress",
        "future", "futuristic", "revolutionary", "cutting-edge", "novel",
        "development", "improvement", "evolution", "next generation",
        "transformative", "modernization", "discovery", "pioneering",
        "groundbreaking", "promising", "visionary", "trendsetting",
        "game-changing", "paradigm shift", "progressive", "emerging",
        "enhancement", "upgrading", "state-of-the-art"
    ]

    c1_sample = cluster_df.sample(n=min(200, len(cluster_df)), random_state=42)
    c2_sample = cluster_2_df.sample(n=min(200, len(cluster_2_df)), random_state=42)

    cluster_1_text = ' '.join(c1_sample['text'].astype(str)).lower()
    cluster_2_text = ' '.join(c2_sample['text'].astype(str)).lower()

    c1_risk_score = sum(cluster_1_text.count(word) for word in surveillance_words)
    c1_innovation_score = sum(cluster_1_text.count(word) for word in innovation_words)

    c2_risk_score = sum(cluster_2_text.count(word) for word in surveillance_words)
    c2_innovation_score = sum(cluster_2_text.count(word) for word in innovation_words)

    c1_words = len(cluster_1_text.split())
    c2_words = len(cluster_2_text.split())

    comparison_data = {
        'Cluster': ['Surveillance (C1)', 'Emerging Tech (C2)'],
        'Risk Language': [c1_risk_score/c1_words*1000 if c1_words > 0 else 0,
                         c2_risk_score/c2_words*1000 if c2_words > 0 else 0],
        'Innovation Language': [c1_innovation_score/c1_words*1000 if c1_words > 0 else 0,
                               c2_innovation_score/c2_words*1000 if c2_words > 0 else 0]
    }

    comparison_df = pd.DataFrame(comparison_data)

    x = range(len(comparison_df))
    width = 0.35

    plt.figure(figsize=(12, 6))
    plt.bar([i - width/2 for i in x], comparison_df['Risk Language'],
            width, label='Risk Language', color='red', alpha=0.7)
    plt.bar([i + width/2 for i in x], comparison_df['Innovation Language'],
            width, label='Innovation Language', color='blue', alpha=0.7)

    plt.xlabel('Coverage Type')
    plt.ylabel('Word Frequency (per 1000 words)')
    plt.title("Wired's Editorial Strategy: Risk vs Innovation Discourse")
    plt.xticks(x, comparison_df['Cluster'])
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

    print("Wired's Editorial Language Patterns:")
    for i, row in comparison_df.iterrows():
        print(f"{row['Cluster']}:")
        print(f"  Risk language intensity: {row['Risk Language']:.2f} per 1000 words")
        print(f"  Innovation language intensity: {row['Innovation Language']:.2f} per 1000 words")
        ratio = row['Innovation Language'] / row['Risk Language'] if row['Risk Language'] > 0 else float('inf')
        print(f"  Innovation/Risk ratio: {ratio:.2f}")

    return comparison_df

print("Starting full analysis of Cluster 2...")
cluster_2_df, cluster_2_analysis = analyze_cluster_2_complete()

print("\n comparing Cluster 1 vs Cluster 2...")
discourse_comparison = compare_cluster_1_vs_2_discourse(cluster_df, cluster_2_df)


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bertopic import BERTopic

# Reload data and model
OUT_DIR = "/content/drive/MyDrive/wired_project"
CSV_PATH = "/content/drive/MyDrive/wired_project/wired_2014_2024_cleaned.csv"

topic_model = BERTopic.load(f"{OUT_DIR}/bertopic_final_model_v4_stopwords")
df = pd.read_csv(CSV_PATH)
docs = df["text"].dropna().astype(str).tolist()
doc_info = topic_model.get_document_info(docs)

print("Data reloaded. Document count:", len(docs))


def analyze_cluster_0_consumer_tech_risks():
    """Cluster 0: Consumer Tech Risk Analysis"""

    print("Cluster 0 Analysis: Consumer Tech Risks")
    print("=" * 60)

    # Topics in cluster 0
    cluster_0_topics = [4, 13, 21, 61]

    cluster_0_df = doc_info[doc_info["Topic"].isin(cluster_0_topics)].copy()
    cluster_0_df["text"] = df.loc[cluster_0_df.index, "text"]
    cluster_0_df["year"] = pd.to_datetime(df.loc[cluster_0_df.index, "date"]).dt.year

    print(f"Total documents: {len(cluster_0_df)}")
    print(f"Number of topics: {len(cluster_0_topics)}")

    # Per-topic analysis
    topic_analysis = {}
    for topic_id in cluster_0_topics:
        topic_docs = cluster_0_df[cluster_0_df['Topic'] == topic_id]
        topic_keywords = topic_model.get_topic(topic_id)

        if len(topic_docs) > 0:
            topic_analysis[topic_id] = {
                'doc_count': len(topic_docs),
                'keywords': topic_keywords[:10],
                'peak_year': topic_docs.groupby('year').size().idxmax()
            }

            print(f"\nTopic {topic_id}: {len(topic_docs)} documents")
            print(f"  Peak year: {topic_analysis[topic_id]['peak_year']}")
            top_5_keywords = [word for word, score in topic_keywords[:5]]
            print(f"  Top keywords: {', '.join(top_5_keywords)}")

    # Create visualizations
    create_topic_keywords_visualization(topic_analysis)
    create_cluster_0_risk_analysis(cluster_0_df, topic_analysis)

    return cluster_0_df, topic_analysis


def create_topic_keywords_visualization(topic_analysis):
    """Create visualization of topic keywords"""

    fig, axes = plt.subplots(2, 2, figsize=(16, 10))
    axes = axes.ravel()

    colors = ['lightcoral', 'lightblue', 'lightgreen', 'gold']
    topic_ids = [4, 13, 21, 61]

    for i, topic_id in enumerate(topic_ids):
        if topic_id in topic_analysis:
            keywords_data = topic_analysis[topic_id]['keywords']
            words = [word for word, score in keywords_data]
            scores = [score for word, score in keywords_data]

            y_pos = range(len(words))
            bars = axes[i].barh(y_pos, scores, color=colors[i], alpha=0.8)

            axes[i].set_yticks(y_pos)
            axes[i].set_yticklabels(words)
            axes[i].invert_yaxis()
            axes[i].set_xlabel('Topic Weight')
            axes[i].set_title(f'Topic {topic_id} ({topic_analysis[topic_id]["doc_count"]} docs)')

            for j, (bar, score) in enumerate(zip(bars, scores)):
                axes[i].text(
                    bar.get_width() + max(scores) * 0.02,
                    bar.get_y() + bar.get_height() / 2,
                    f'{score:.3f}',
                    ha='left',
                    va='center',
                    fontsize=8
                )

            axes[i].set_xlim(0, max(scores) * 1.3)
            axes[i].grid(axis='x', alpha=0.3)
        else:
            axes[i].text(
                0.5, 0.5, f'Topic {topic_id}\nNo data',
                transform=axes[i].transAxes, ha='center', va='center'
            )

    plt.suptitle('Cluster 0: Consumer Tech Risk Topics - Keyword Distribution')
    plt.tight_layout()
    plt.show()


def create_cluster_0_risk_analysis(cluster_df, topic_analysis):
    """Create risk analysis visualizations"""

    fig, axes = plt.subplots(2, 2, figsize=(15, 10))

    # 1. Topic document distribution
    topic_ids = list(topic_analysis.keys())
    doc_counts = [topic_analysis[tid]['doc_count'] for tid in topic_ids]

    bars = axes[0, 0].bar(
        range(len(topic_ids)),
        doc_counts,
        color=['lightcoral', 'lightblue', 'lightgreen', 'gold']
    )
    axes[0, 0].set_title('Topic Distribution')
    axes[0, 0].set_xlabel('Topic ID')
    axes[0, 0].set_ylabel('Document Count')
    axes[0, 0].set_xticks(range(len(topic_ids)))
    axes[0, 0].set_xticklabels([f'T{tid}' for tid in topic_ids])

    for bar, count in zip(bars, doc_counts):
        axes[0, 0].text(
            bar.get_x() + bar.get_width() / 2,
            bar.get_height() + max(doc_counts) * 0.01,
            str(count),
            ha='center',
            va='bottom'
        )

    # 2. Yearly trend
    yearly_counts = cluster_df.groupby('year').size().reset_index(name='count')
    axes[0, 1].plot(
        yearly_counts['year'],
        yearly_counts['count'],
        marker='o',
        linewidth=2,
        color='red'
    )
    axes[0, 1].set_title('Coverage Over Time')
    axes[0, 1].set_xlabel('Year')
    axes[0, 1].set_ylabel('Articles')
    axes[0, 1].grid(True, alpha=0.3)

    # 3. Topic trends over time
    for topic_id in topic_ids:
        topic_docs = cluster_df[cluster_df['Topic'] == topic_id]
        if len(topic_docs) > 0:
            topic_yearly = topic_docs.groupby('year').size()
            axes[1, 0].plot(
                topic_yearly.index,
                topic_yearly.values,
                marker='o',
                label=f'Topic {topic_id}'
            )

    axes[1, 0].set_title('Individual Topic Trends')
    axes[1, 0].set_xlabel('Year')
    axes[1, 0].set_ylabel('Articles')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)

    # 4. Relative importance pie chart
    axes[1, 1].pie(
        doc_counts,
        labels=[f'Topic {tid}' for tid in topic_ids],
        autopct='%1.1f%%',
        colors=['lightcoral', 'lightblue', 'lightgreen', 'gold']
    )
    axes[1, 1].set_title('Relative Topic Importance')

    plt.tight_layout()
    plt.show()


# Run analysis
cluster_0_df, cluster_0_analysis = analyze_cluster_0_consumer_tech_risks()

In [ ]:
# Cluster 3 Analysis

def analyze_cluster_3_corporate_industry_risks():
    """Cluster 3: Multi-dimensional Analysis of Corporate and Industry Risks"""

    print("Cluster 3 Analysis: Corporate and Industry Risk Coverage")
    print("=" * 65)

    # Topics in cluster 3
    cluster_3_topics = [6, 9, 10, 16, 18, 19, 20, 29, 30, 31, 35, 39, 40, 41, 43, 50, 60]

    cluster_3_df = doc_info[doc_info["Topic"].isin(cluster_3_topics)].copy()
    cluster_3_df["text"] = df.loc[cluster_3_df.index, "text"]
    cluster_3_df["year"] = pd.to_datetime(df.loc[cluster_3_df.index, "date"]).dt.year
    cluster_3_df["month"] = pd.to_datetime(df.loc[cluster_3_df.index, "date"]).dt.month

    print(f"Total documents: {len(cluster_3_df)}")
    print(f"Topics analyzed: {len(cluster_3_topics)}")

    # Analyses
    editorial_stance_analysis = analyze_wired_editorial_stance_shift(cluster_3_df)
    coverage_depth_analysis = analyze_coverage_depth_patterns(cluster_3_df)
    geographic_risk_analysis = analyze_geographic_risk_distribution(cluster_3_df)

    # Visualizations
    create_cluster_3_diverse_visualizations(
        cluster_3_df,
        editorial_stance_analysis,
        coverage_depth_analysis,
        geographic_risk_analysis
    )

    return cluster_3_df


def analyze_wired_editorial_stance_shift(cluster_df):
    """Analyze Wired's editorial stance shift on corporate tech"""

    critical_language = [
        "problematic", "concerning", "questionable", "controversial", "criticism",
        "backlash", "flawed", "biased", "unethical", "unfair", "dangerous",
        "harmful", "shortcomings", "limitations", "opposition", "challenge",
        "resistance", "disputed", "skeptical", "negative", "troubling",
        "scandal", "violation", "abuse", "failure", "collapse", "corruption",
        "misleading", "inaccurate", "doubtful", "critical", "condemned"
    ]

    neutral_language = [
        "analysis", "examination", "assessment", "evaluation", "review",
        "overview", "report", "discussion", "description", "explanation",
        "study", "investigation", "comparison", "account", "summary",
        "interpretation", "findings", "observations", "consideration",
        "neutral", "outline", "exploration", "presentation", "profiling"
    ]

    supportive_language = [
        "innovative", "pioneering", "groundbreaking", "impressive", "successful",
        "leading", "promising", "cutting-edge", "revolutionary", "progressive",
        "remarkable", "visionary", "transformative", "breakthrough", "extraordinary",
        "outstanding", "excellent", "superior", "notable", "celebrated",
        "influential", "inspiring", "acclaimed", "renowned", "commendable",
        "trailblazing", "advanced", "state-of-the-art", "game-changing", "exceptional"
    ]

    stance_evolution = {}

    for year in sorted(cluster_df['year'].unique()):
        year_sample = cluster_df[cluster_df['year'] == year].sample(
            n=min(30, len(cluster_df[cluster_df['year'] == year])),
            random_state=42
        )
        year_text = ' '.join(year_sample['text']).lower()
        total_words = len(year_text.split())

        critical_score = sum(year_text.count(word) for word in critical_language)
        neutral_score = sum(year_text.count(word) for word in neutral_language)
        supportive_score = sum(year_text.count(word) for word in supportive_language)

        total_stance = critical_score + neutral_score + supportive_score

        if total_stance > 0:
            stance_evolution[year] = {
                'critical_ratio': critical_score / total_stance,
                'neutral_ratio': neutral_score / total_stance,
                'supportive_ratio': supportive_score / total_stance,
                'stance_intensity': total_stance / total_words * 1000 if total_words > 0 else 0
            }

    print("WIRED'S EDITORIAL STANCE EVOLUTION:")
    recent_critical = stance_evolution[2024]['critical_ratio'] if 2024 in stance_evolution else 0
    early_critical = stance_evolution[2014]['critical_ratio'] if 2014 in stance_evolution else 0

    trend = "more critical" if recent_critical > early_critical else "less critical"
    print(f"  Editorial trend: {trend} of corporate tech")
    print(f"  2014 critical stance: {early_critical:.1%}")
    print(f"  2024 critical stance: {recent_critical:.1%}")

    return stance_evolution


def analyze_coverage_depth_patterns(cluster_df):
    """Analyze patterns of Wired's reporting depth and complexity"""

    depth_indicators = {
        'investigative': ['investigation', 'documents reveal', 'sources say', 'obtained by', 'internal'],
        'analytical': ['analysis', 'implications', 'consequences', 'broader context', 'underlying'],
        'surface': ['announced', 'released', 'launched', 'unveiled', 'introduced']
    }

    depth_patterns = {}

    for year in sorted(cluster_df['year'].unique()):
        year_docs = cluster_df[cluster_df['year'] == year]
        avg_length = year_docs['text'].apply(len).mean()
        year_text = ' '.join(year_docs['text'].head(50)).lower()

        investigative_signals = sum(year_text.count(phrase) for phrase in depth_indicators['investigative'])
        analytical_signals = sum(year_text.count(phrase) for phrase in depth_indicators['analytical'])
        surface_signals = sum(year_text.count(phrase) for phrase in depth_indicators['surface'])

        total_signals = investigative_signals + analytical_signals + surface_signals

        depth_patterns[year] = {
            'avg_article_length': avg_length,
            'investigative_ratio': investigative_signals / total_signals if total_signals > 0 else 0,
            'analytical_ratio': analytical_signals / total_signals if total_signals > 0 else 0,
            'surface_ratio': surface_signals / total_signals if total_signals > 0 else 0
        }

    print("COVERAGE DEPTH ANALYSIS:")
    avg_length_trend = (
        (depth_patterns[2024]['avg_article_length'] - depth_patterns[2014]['avg_article_length'])
        / depth_patterns[2014]['avg_article_length'] * 100
        if 2014 in depth_patterns and 2024 in depth_patterns else 0
    )
    print(f"  Article length trend: {avg_length_trend:+.1f}% change from 2014 to 2024")

    return depth_patterns


def analyze_geographic_risk_distribution(cluster_df):
    """Analyze geographic distribution of corporate risk coverage"""

    regions = {
        'Silicon Valley': ['silicon valley', 'san francisco', 'palo alto', 'menlo park', 'cupertino'],
        'Seattle Tech': ['seattle', 'redmond', 'bellevue'],
        'International': ['china', 'europe', 'london', 'berlin', 'tokyo'],
        'East Coast': ['new york', 'boston', 'washington dc']
    }

    geo_distribution = {}

    for year in sorted(cluster_df['year'].unique()):
        year_text = ' '.join(cluster_df[cluster_df['year'] == year]['text'].head(30)).lower()

        year_geo = {}
        for region, locations in regions.items():
            mentions = sum(year_text.count(location) for location in locations)
            year_geo[region] = mentions

        geo_distribution[year] = year_geo

    print("GEOGRAPHIC FOCUS ANALYSIS:")
    sv_dominance = (
        sum(geo_distribution[year]['Silicon Valley'] for year in geo_distribution)
        / sum(sum(year_data.values()) for year_data in geo_distribution.values())
        if geo_distribution else 0
    )
    print(f"  Silicon Valley coverage dominance: {sv_dominance:.1%}")

    return geo_distribution


def create_cluster_3_diverse_visualizations(cluster_df, stance_data, depth_data, geo_data):
    """Create diverse visualizations for Cluster 3"""

    fig = plt.figure(figsize=(20, 12))

    # 1. Editorial stance ratios
    ax1 = plt.subplot(2, 4, 1)
    years = list(stance_data.keys())
    critical_ratios = [stance_data[year]['critical_ratio'] for year in years]
    neutral_ratios = [stance_data[year]['neutral_ratio'] for year in years]
    supportive_ratios = [stance_data[year]['supportive_ratio'] for year in years]

    ax1.fill_between(years, critical_ratios, color='red', alpha=0.3, label='Critical')
    ax1.fill_between(years, neutral_ratios, color='gray', alpha=0.3, label='Neutral')
    ax1.fill_between(years, supportive_ratios, color='green', alpha=0.3, label='Supportive')
    ax1.set_title("Wired's Editorial Stance Evolution")
    ax1.legend()
    ax1.set_ylabel('Stance Ratio')

    # 2. Coverage depth (pie chart of recent year)
    ax2 = plt.subplot(2, 4, 2)
    recent_depth = depth_data[2024] if 2024 in depth_data else depth_data[max(depth_data.keys())]

    categories = ['Investigative', 'Analytical', 'Surface']
    values = [
        recent_depth['investigative_ratio'],
        recent_depth['analytical_ratio'],
        recent_depth['surface_ratio']
    ]

    ax2.pie(values, labels=categories, autopct='%1.1f%%',
            colors=['darkred', 'orange', 'lightblue'])
    ax2.set_title('Current Coverage Depth Mix')

    # 3. Geographic distribution
    ax3 = plt.subplot(2, 4, (3, 4))
    geo_years = list(geo_data.keys())

    sv_data = [geo_data[year]['Silicon Valley'] for year in geo_years]
    seattle_data = [geo_data[year]['Seattle Tech'] for year in geo_years]
    intl_data = [geo_data[year]['International'] for year in geo_years]
    east_data = [geo_data[year]['East Coast'] for year in geo_years]

    ax3.stackplot(
        geo_years, sv_data, seattle_data, intl_data, east_data,
        labels=['Silicon Valley', 'Seattle Tech', 'International', 'East Coast'],
        colors=['gold', 'lightgreen', 'lightcoral', 'lightblue'],
        alpha=0.8
    )
    ax3.set_title('Geographic Focus of Corporate Risk Coverage')
    ax3.legend(loc='upper left', fontsize=8)
    ax3.set_ylabel('Geographic Mentions')

    # 4. Article length trend
    ax4 = plt.subplot(2, 4, 5)
    depth_years = list(depth_data.keys())
    avg_lengths = [depth_data[year]['avg_article_length'] for year in depth_years]

    ax4.plot(depth_years, avg_lengths, 'bo-', linewidth=2, markersize=6)
    ax4.fill_between(depth_years, avg_lengths, alpha=0.3, color='blue')
    ax4.set_title('Article Length Trend')
    ax4.set_ylabel('Avg Characters per Article')
    ax4.grid(True, alpha=0.3)

    # 5. Monthly coverage heatmap
    ax5 = plt.subplot(2, 4, 6)
    monthly_data = cluster_df.groupby(['year', 'month']).size().unstack(fill_value=0)
    if len(monthly_data) > 0:
        im = ax5.imshow(monthly_data.values, cmap='YlOrRd', aspect='auto')
        ax5.set_title('Monthly Coverage Intensity')
        ax5.set_xlabel('Month')
        ax5.set_ylabel('Year')
        ax5.set_xticks(range(12))
        ax5.set_xticklabels(['J', 'F', 'M', 'A', 'M', 'J', 'J', 'A', 'S', 'O', 'N', 'D'])
        ax5.set_yticks(range(len(monthly_data.index)))
        ax5.set_yticklabels(monthly_data.index)

    # 6. Stance intensity vs coverage volume
    ax6 = plt.subplot(2, 4, 7)
    yearly_counts = cluster_df.groupby('year').size()
    stance_intensities = [stance_data[year]['stance_intensity'] for year in years if year in yearly_counts.index]
    coverage_volumes = [yearly_counts[year] for year in years if year in yearly_counts.index]

    scatter = ax6.scatter(
        stance_intensities,
        coverage_volumes,
        c=[year for year in years if year in yearly_counts.index],
        cmap='viridis', s=60, alpha=0.7
    )
    ax6.set_xlabel('Editorial Stance Intensity')
    ax6.set_ylabel('Coverage Volume')
    ax6.set_title('Stance vs Volume Correlation')
    plt.colorbar(scatter, ax=ax6, label='Year')

    # 7. Composite quality index
    ax7 = plt.subplot(2, 4, 8)
    composite_scores = []
    composite_years = []

    for year in years:
        if year in depth_data and year in yearly_counts.index:
            critical_weight = stance_data[year]['critical_ratio'] * 0.4
            depth_weight = depth_data[year]['investigative_ratio'] * 0.3
            volume_weight = (yearly_counts[year] / yearly_counts.max()) * 0.3

            composite = critical_weight + depth_weight + volume_weight
            composite_scores.append(composite)
            composite_years.append(year)

    ax7.bar(composite_years, composite_scores, color='purple', alpha=0.7)
    ax7.set_title('Wired Corporate Coverage\nQuality Index')
    ax7.set_ylabel('Composite Score')
    ax7.tick_params(axis='x', rotation=45)

    plt.tight_layout()
    plt.show()


# Run Cluster 3 analysis
cluster_3_df = analyze_cluster_3_corporate_industry_risks()

In [ ]:
# Cluster 4
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

def analyze_cluster_4_policy(topic_model, doc_info, df):
    """Cluster 4: Technology Policy and Communication Systems"""

    # Topics in cluster 4
    cluster_4_topics = [14, 18, 56, 64]
    cluster_4_df = doc_info[doc_info["Topic"].isin(cluster_4_topics)].copy()
    cluster_4_df["text"] = df.loc[cluster_4_df.index, "text"]
    cluster_4_df["year"] = pd.to_datetime(df.loc[cluster_4_df.index, "date"]).dt.year

    print("Cluster 4: Technology Policy and Communication Systems")
    print("=" * 70)
    print(f"Total documents: {len(cluster_4_df)}")
    print(f"Topics: {cluster_4_topics}")

    # Keyword distributions
    fig, axes = plt.subplots(1, len(cluster_4_topics), figsize=(16, 4))
    for i, tid in enumerate(cluster_4_topics):
        kws = topic_model.get_topic(tid)[:10]
        words, scores = zip(*kws)
        axes[i].barh(words[::-1], scores[::-1], color="teal")
        axes[i].set_title(f"Topic {tid}")
    plt.suptitle("Technology Policy and Communication Systems - Keyword Distributions")
    plt.tight_layout()
    plt.show()

    # Temporal trends (overall + per topic)
    yearly_counts = cluster_4_df.groupby("year").size()
    plt.figure(figsize=(8, 5))
    plt.plot(yearly_counts.index, yearly_counts.values, marker="o", color="darkred")
    plt.title("Technology Policy and Communication Systems - Coverage Over Time")
    plt.xlabel("Year"); plt.ylabel("Articles")
    plt.grid(alpha=0.3)
    plt.show()

    plt.figure(figsize=(10, 6))
    for tid in cluster_4_topics:
        topic_docs = cluster_4_df[cluster_4_df["Topic"] == tid]
        if not topic_docs.empty:
            topic_yearly = topic_docs.groupby("year").size()
            plt.plot(topic_yearly.index, topic_yearly.values, marker="o", label=f"Topic {tid}")
    plt.title("Technology Policy and Communication Systems - Individual Topic Trends")
    plt.xlabel("Year"); plt.ylabel("Articles")
    plt.legend(); plt.grid(alpha=0.3)
    plt.show()

    # Document length distribution
    cluster_4_df["doc_length"] = cluster_4_df["text"].str.len()
    plt.figure(figsize=(8, 5))
    cluster_4_df["doc_length"].hist(bins=30, color="orange", edgecolor="black")
    plt.axvline(cluster_4_df["doc_length"].mean(), color="red", linestyle="--",
                label=f"Mean length: {cluster_4_df['doc_length'].mean():.0f}")
    plt.title("Document Length Distribution - Technology Policy and Communication Systems")
    plt.xlabel("Characters"); plt.ylabel("Frequency")
    plt.legend(); plt.show()

    # Top entities (NER)
    try:
        import spacy
        nlp = spacy.load("en_core_web_sm")
        texts = cluster_4_df["text"].sample(min(200, len(cluster_4_df)), random_state=42).tolist()
        ents = []
        for doc in nlp.pipe(texts, disable=["parser", "tagger"]):
            ents.extend([ent.text for ent in doc.ents if ent.label_ in ["GPE", "ORG"]])
        top_ents = Counter(ents).most_common(15)
        ents_df = pd.DataFrame(top_ents, columns=["Entity", "Count"])
        ents_df.plot(kind="barh", x="Entity", y="Count", figsize=(8, 5), color="steelblue")
        plt.title("Top Entities in Technology Policy and Communication Systems")
        plt.show()
    except:
        print("spaCy not installed or model not available - skip NER.")

    return cluster_4_df


# Run Cluster 4 analysis
cluster_4_df = analyze_cluster_4_policy(topic_model, doc_info, df)
